In [ ]:
#IMPORT PYTHON PACKAGES
import seaborn as sns
import pandas as pd
import scipy as sp
import scipy.integrate
import numpy  as np
import copy
import matplotlib.pylab as plt
from matplotlib import rcParams
from matplotlib.ticker import FormatStrFormatter
import time
import datetime
import subprocess
import itertools
from itertools import groupby, repeat, islice
import math as math
import decimal as d
from decimal import *


#Plot settings
sns.set_style("whitegrid", rc={"axes.edgecolor": "k", "axes.linewidth":2.})

sns.set_style("ticks", {"xtick.major.size":8,"ytick.major.size":8})


sns.set_context("notebook",rc={"grid.linewidth": 0, 
                            "font.family":"Helvetica", "axes.labelsize":24.,"xtick.labelsize":24., 
                            "ytick.labelsize":24., "legend.fontsize":18.})

color_blind_safe = sns.color_palette("colorblind", 10) 

colors = color_blind_safe

In [ ]:

filename = ['./profile_geE200','profile_geE400','profile_geE600',
           'profile_geE800','profile_geE1000'] #


MP_Profile = []; MP_Ave = []; MP_LastN = []; 

for simNo in range(0,len(filename)):
    subprocess.call(r"perl -p -e 's/^[\ \t]+|[\ \t]+$//g' %s > %s_new" % (filename[simNo], filename[simNo]), shell=True) # remove trailling white spaces
    subprocess.call("mv %s_new %s" % (filename[simNo], filename[simNo]), shell=True)
    subprocess.call("tr -s ' ' < %s > %s_tmp " % (filename[simNo], filename[simNo]), shell=True) #squeeze spaces
    subprocess.call("mv %s_tmp %s" % (filename[simNo], filename[simNo]), shell=True)

    profile = []; N = 800;
    with open(filename[simNo], 'r') as f:
        while True:
            lines = list(itertools.islice(f, 1, N+1))
            for line in lines:
                profile.append(line.split(' '))
            if not lines: 
                break
                
    lastN = profile[-N:] 
    lastAve = profile[-3200:]
    
    MP_Profile.append(profile)
    MP_LastN.append(lastN)
    MP_Ave.append(lastAve)
    
MP_Profile = np.array(MP_Profile, dtype=float) # for plotting 3D graphs

MP_LastN = np.array(MP_LastN, dtype=float) # for calculating the thermal conductivity



MP_Ave = np.array(MP_Ave, dtype=float) # for averaging the data to calculate the thermal conductivity

Thermo_Data = np.array([np.loadtxt('./Thermo_geE200'),np.loadtxt('./Thermo_geE400'),np.loadtxt('./Thermo_geE600'),
                       np.loadtxt('./Thermo_geE800'),np.loadtxt('./Thermo_geE1000')], dtype = object)

labels = ['ge200', 'ge400', 'ge600','ge800','ge1000','ge1500','ge2000'] 


for simNo in range(0, len(MP_Profile)):
    Coordinates = MP_Profile[simNo,:,1]*(Thermo_Data[simNo,-1,9])
    Temperature = MP_Profile[simNo,:,3]
    
fig = plt.figure(figsize=(12,8))
for k in range(0, len(MP_Profile)):
    plt.plot(Coordinates[:], Temperature[:], color=colors[k], label=labels[k])
    
plt.xlabel(r'Position in Supercell ($\AA$)')
plt.ylabel('Temperature (K)')
plt.title('Muller-Plathe Temperature Gradient', fontsize = 24)
plt.show

# Load the data that will be averaged to calculate the slopes 
Temperature = []; Coordinate = []; dX_Thermo = [];

for simNo in range(0, len(filename)):
    
    dX_ = Thermo_Data[simNo,-1,9]
    dX_Thermo.append(dX_)
    
    for i in range(0,800):
        Temperature_ = np.mean([MP_Ave[simNo,i,3],MP_Ave[simNo,i+800,3],MP_Ave[simNo,i+1600,3],MP_Ave[simNo,i+2400,3]])   
        Temperature.append(Temperature_)
        Coordinate_ = np.mean([MP_Ave[simNo,i,1],MP_Ave[simNo,i+800,1],MP_Ave[simNo,i+1600,1],MP_Ave[simNo,i+2400,1]])*dX_
        Coordinate.append(Coordinate_)
    
Temperature = np.array([Temperature[0:800],Temperature[800:1600],Temperature[1600:2400],
                        Temperature[2400:3200],Temperature[3200:4000]], dtype=float) 
                       
Coordinate = np.array([Coordinate[0:800],Coordinate[800:1600],Coordinate[1600:2400],
                      Coordinate[2400:3200],Coordinate[3200:4000]], dtype=float) 
                      


excluded_points1 = 230 # excluded points from the beginning
excluded_points2 = 40 # excluded points from the end
excluded_points3 = 20
excluded_points4 = 230 # from the end of the mp.data file

slope2_mean = []; slope2 = []; slope1_mean = []; slope1 = []; average_temp_PrstG = [];

# Step Temp E_pair PotEng KinEng TotEng Temp Press Volume Lx Ly Lz f_3 v_tdiff
for simNo in range(0,len(filename)):
    
    dTemp = Temperature[simNo,:]
    X = dX_
    dX = Coordinate[simNo,:] #*X # normalized distance
    
    T_fit_left = np.polyfit(dX[excluded_points1:400-excluded_points2], dTemp[excluded_points1:400-excluded_points2], 1)
    T_fit_right = -np.polyfit(dX[400+excluded_points2:-excluded_points1], dTemp[400+excluded_points2:-excluded_points1], 1)
    
    slope2.append([T_fit_left[0], T_fit_right[0]])
    mean_slope = np.mean([abs(T_fit_left[0]), abs(T_fit_right[0])], axis = 0)
    slope2_mean.append(mean_slope)
    B_fit_left2 = np.polyfit(dX[excluded_points3:400-excluded_points4], dTemp[excluded_points3:400-excluded_points4], 1)
    B_fit_right2 = -np.polyfit(dX[400+excluded_points4:-excluded_points3], dTemp[400+excluded_points4:-excluded_points3], 1)   
    slope1.append([B_fit_left2[0], B_fit_right2[0]])
    mean_slope = np.mean([abs(B_fit_left2[0]), abs(B_fit_right2[0])], axis = 0)
    slope1_mean.append(mean_slope)
    average_temp = np.mean(dTemp)
    average_temp_PrstG.append(average_temp)

Slope1 = np.array(slope1)
Slope_mean1 = np.array(slope1_mean)

Slope2 = np.array(slope2)
Slope_mean2 = np.array(slope2_mean)

Slope_mean = np.array([np.mean([Slope_mean1[0], Slope_mean2[0]], axis=0)]) #individual
#Slope_mean = np.array([np.mean([Slope_mean1[0], Slope_mean2[0]], axis=0), np.mean([Slope_mean1[1], Slope_mean2[1]], axis =0), np.mean([Slope_mean1[2], Slope_mean2[2]], axis=0)])

Slope_ave = np.array([Slope_mean[0]*dX_Thermo[0]]) #individual
#Slope_ave = np.array([Slope_mean[0]*dX_Thermo[0], Slope_mean[1]*dX_Thermo[1], Slope_mean[2]*dX_Thermo[2]])

# Calculate the heat flux and the thermal conductivity
eV2J  = 1.6e-19;
Ang2m =   1e-10;
ps2s  =   1e-12;
conversion = (eV2J/(Ang2m*ps2s))

s_values = []
ThermalsprstG = []
dt = 0.0005 # picoseconds

# Step Temp E_pair PotEng KinEng TotEng Temp Press Volume Lx Ly Lz f_3 v_tdiff
for simNo in range(0, len(Thermo_Data)):
    
    Steps = Thermo_Data[simNo,-1,0]
    Time = Steps*dt # picoseconds
    
    dx = Thermo_Data[simNo,-1,9]
    dy = Thermo_Data[simNo,-1,10]
    dz = Thermo_Data[simNo,-1,11]
    Area = dy*dz
    
    #DeltaE = Thermo_Data[simNo,-1,-2] # E_pair
    #TotalE = Thermo_Data[simNo,-1,-5] # Total energy
    KinE = Thermo_Data[simNo,-1,12] # in log_lammps, f_3 
    
    Thermal_left2 = ((KinE)/(2*Time*Area*Slope2[simNo,0]))*conversion #top left
    Thermal_right2 = ((KinE)/(2*Time*Area*Slope2[simNo,1]))*conversion #top right
    
    Thermal_left1 = ((KinE)/(2*Time*Area*Slope1[simNo,0]))*conversion #bottom left
    Thermal_right1 = ((KinE)/(2*Time*Area*Slope1[simNo,1]))*conversion #bottom right
    
    thermal2 = np.concatenate((Thermal_left1, Thermal_left2), axis = None)
    thermal1 = np.concatenate((Thermal_right1, Thermal_right2), axis = None)
    
    thermal2_ave = ((KinE)/(2*Time*Area*Slope_mean2[simNo]))*conversion #top
    thermal1_ave = ((KinE)/(2*Time*Area*Slope_mean1[simNo]))*conversion #bottom
    
    #thermal_ave = ((KinE)/(2*Time*Area*Slope_ave[simNo]))*conversion
    thermal_ave = (thermal2_ave+thermal1_ave)/2
   
    getcontext().prec = 10
    std_top = Decimal(np.std([Thermal_left2, Thermal_right2], dtype=np.float64))
    SE_left = std_top/Decimal(math.sqrt(2))
   
    std_bottom = Decimal(np.std([Thermal_left1, Thermal_right1], dtype=np.float64))
    SE_right = std_bottom/Decimal(math.sqrt(2))
    
    SE = math.sqrt(((SE_left/2)**2) + ((SE_right/2)**2))
    
    Therm_val = [Thermal_left1,Thermal_left2,Thermal_right1,Thermal_right2]
    Ttherm = ['%.6f' % elem for elem in Therm_val]
    ThermalsprstG.append(Ttherm)
    
    TC_values = [thermal2_ave, thermal1_ave, thermal_ave, SE]
    val = ['%.6f' % elem for elem in TC_values]
    s_values.append(val)
    

therm_average_prstG = []
for item in s_values:
    therm_average_prstG.append(item[2])

therm_average_prstG = []
for item in s_values:
    therm_average_prstG.append(np.array(float(item[2])))

#projection stuff
size_prstG = [200.0,400.0,600.0,800.0,1000.0]

size_prstG = np.array(size_prstG)
Tc = [float(s_values[0][2]),float(s_values[1][2]),float(s_values[2][2]),float(s_values[3][2]),float(s_values[4][2])]
Tc = np.array(Tc)
inv_size_prstG = np.reciprocal(size_prstG)
inv_Tc = np.reciprocal(Tc)

fig = plt.figure(figsize=(12,8))

plt.plot(size_prstG[:], Tc[:],'o-')


plt.xlabel(r'Length', fontdict={'fontsize':27})
plt.ylabel(r'k', fontdict={'fontsize':27})
plt.title(r'Projection of Convergence for Pristine Ge', fontdict={'fontsize':27})


fig = plt.figure(figsize=(12,8))

x1 = np.array(size_prstG)
mn = min(x1)
mx = max(x1)

y1 = np.array(Tc)

z1 = np.polyfit(x1, y1, 1) #np.polyfit(x, y, degree of polynomial), degree =1,2,3 is linear, quadratic, cubic
z2 = np.polyfit(x1, y1, 2)
z3 = np.polyfit(x1, y1, 3)

xp = np.linspace(-0.001, mx+0.001, 6)

p1 = np.poly1d(z1)
p2 = np.poly1d(z2)
p3 = np.poly1d(z3)
#pz = np.poly1d(np.polyfit(x, y, 3)) #same as p3

plt.plot(x1, y1, 'o-', color ='black', label ='OG')
plt.plot(xp, p1(xp), 's', label = 'linear')
plt.plot(xp, p2(xp), 's', label = 'quadratic')
plt.plot(xp, p3(xp), 's', label ='cubic') 

plt.axvline(x=125.0, color=color_blind_safe[3])

plt.legend(loc="lower right")

plt.xlim([0.0,2000])
plt.ylim([0.0,20])

plt.xlabel(r'Length', fontdict={'fontsize':27})
plt.ylabel(r'k', fontdict={'fontsize':27})
plt.title(r'Projection of Convergence for Pristine Ge', fontdict={'fontsize':27})

fig = plt.figure(figsize=(12,8))

x_2 = np.array(size_prstG)
mn = min(x_2)
mx = max(x_2)

y_2 = np.array(Tc)
xpp = np.linspace(-0.0005,mx, 6)

plt.axvline(x=125.0, color=color_blind_safe[3])


z3 = np.polyfit(x_2, y_2, 3)
p3 = np.poly1d(z3)
#pz = np.poly1d(np.polyfit(x, y, 3)) #same as p3

s = p3(1);
i = p3(2)    #The polyfit function for a linear (polynomial order 1) fit returns the slope as the first parameter
                    #and the intercept as the second parameter, so the output vector is [slope, intercept]. 
                    #It is like any other vector, so choose the one you want by indexing into it.
n = p3(0)
slope_intercept = np.polyfit(x_2,y_2,3)

plt.plot(x_2,y_2, 'o-', label ='OG')
plt.plot(xpp, p3(xp), 's', label ='cubic') 
plt.legend(loc="lower right")

plt.xlabel(r'Length', fontdict={'fontsize':27})
plt.ylabel(r'k', fontdict={'fontsize':27})
plt.title(r'Projection of Convergence for Pristine Ge cubic', fontdict={'fontsize':27})


#linear fit in linear region alone
fig = plt.figure(figsize=(12,8))

x2 = size_prstG
x2 = np.array(x2)
y2 = Tc
y2 = np.array(y2)


z = np.polyfit(x2, y2, 1) 
xp = np.linspace(-0.001, mx+0.001, 3)

p1 = np.poly1d(z)

slope_intercept = np.polyfit(x2,y2,1)

plt.plot(x2, y2, 'o-', color ='black', label ='OG')
plt.plot(xp, p1(xp), 's', label = 'linear')

plt.axvline(x=0, color=color_blind_safe[3])

plt.legend(loc="lower right")

plt.xlabel(r'Length', fontdict={'fontsize':27})
plt.ylabel(r'k', fontdict={'fontsize':27})
plt.title(r'Projection of Convergence for linear Pristine Ge', fontdict={'fontsize':27})


#Pristine Ge values

SE200b_left = np.std([float(ThermalsprstG[0][0]), float(ThermalsprstG[0][2])], dtype=np.float64)
SE200_left = SE200b_left/math.sqrt(2)
SE200b_right = np.std([float(ThermalsprstG[0][1]), float(ThermalsprstG[0][3])], dtype=np.float64)
SE200_right = SE200b_right/math.sqrt(2)
standard_error200b_prstG = math.sqrt(((SE200_left/2)**2) + ((SE200_right/2)**2))


SE400b_left = np.std([float(ThermalsprstG[1][0]), float(ThermalsprstG[1][2])], dtype=np.float64)
SE400_left = SE400b_left/math.sqrt(2)
SE400b_right = np.std([float(ThermalsprstG[1][1]), float(ThermalsprstG[1][3])], dtype=np.float64)
SE400_right = SE400b_right/math.sqrt(2)
standard_error400b_prstG = math.sqrt(((SE400_left/2)**2) + ((SE400_right/2)**2))

SE600b_left = np.std([float(ThermalsprstG[2][0]), float(ThermalsprstG[2][2])], dtype=np.float64)
SE600_left = SE600b_left/math.sqrt(2)
SE600b_right = np.std([float(ThermalsprstG[2][1]), float(ThermalsprstG[2][3])], dtype=np.float64)
SE600_right = SE600b_right/math.sqrt(2)
standard_error600b_prstG = math.sqrt(((SE600_left/2)**2) + ((SE600_right/2)**2))


SE800b_left = np.std([float(ThermalsprstG[3][0]), float(ThermalsprstG[3][2])], dtype=np.float64)
SE800_left = SE800b_left/math.sqrt(2)
SE800b_right = np.std([float(ThermalsprstG[3][1]), float(ThermalsprstG[3][3])], dtype=np.float64)
SE800_right = SE800b_right/math.sqrt(2)
standard_error800b_prstG = math.sqrt(((SE800_left/2)**2) + ((SE800_right/2)**2))

SE1000b_left = np.std([float(ThermalsprstG[4][0]), float(ThermalsprstG[4][2])], dtype=np.float64)
SE1000_left = SE1000b_left/math.sqrt(2)
SE1000b_right = np.std([float(ThermalsprstG[4][1]), float(ThermalsprstG[4][3])], dtype=np.float64)
SE1000_right = SE1000b_right/math.sqrt(2)
standard_error1000b_prstG = math.sqrt(((SE1000_left/2)**2) + ((SE1000_right/2)**2))

yerr200b_prstG = standard_error200b_prstG
yerr400b_prstG = standard_error400b_prstG
yerr600b_prstG = standard_error600b_prstG
yerr800b_prstG = standard_error800b_prstG
yerr1000b_prstG = standard_error1000b_prstG


In [ ]:
filename = ['./profile_siE200', 'profile_siE400','profile_siE600','profile_siE800','profile_siE1000']#
           


MP_Profile = []; MP_Ave = []; MP_LastN = []; 

for simNo in range(0,len(filename)):
    subprocess.call(r"perl -p -e 's/^[\ \t]+|[\ \t]+$//g' %s > %s_new" % (filename[simNo], filename[simNo]), shell=True) # remove trailling white spaces
    subprocess.call("mv %s_new %s" % (filename[simNo], filename[simNo]), shell=True)
    subprocess.call("tr -s ' ' < %s > %s_tmp " % (filename[simNo], filename[simNo]), shell=True) #squeeze spaces
    subprocess.call("mv %s_tmp %s" % (filename[simNo], filename[simNo]), shell=True)

    profile = []; N = 800;
    with open(filename[simNo], 'r') as f:
        while True:
            lines = list(itertools.islice(f, 1, N+1))
            for line in lines:
                profile.append(line.split(' '))
            if not lines: 
                break
                
    lastN = profile[-N:] 
    lastAve = profile[-3200:]

    MP_Profile.append(profile)
    MP_LastN.append(lastN)
    MP_Ave.append(lastAve)
    
MP_Profile = np.array(MP_Profile, dtype=float) # for plotting 3D graphs

MP_LastN = np.array(MP_LastN, dtype=float) # for calculating the thermal conductivity

MP_Ave = np.array(MP_Ave, dtype=float) # for averaging the data to calculate the thermal conductivity

Thermo_Data = np.array([np.loadtxt('./Thermo_siE200'),np.loadtxt('./Thermo_siE400'),np.loadtxt('./Thermo_siE600'),
                       np.loadtxt('./Thermo_siE800'),np.loadtxt('./Thermo_siE1000')], dtype = object) #, ,
                        
labels = ['si200', 'si400', 'si600','si800','si1000','si1500','si2000'] 


for simNo in range(0, len(MP_Profile)):
    Coordinates = MP_Profile[simNo,:,1]*(Thermo_Data[simNo,-1,9])
    Temperature = MP_Profile[simNo,:,3]

fig = plt.figure(figsize=(12,8))
for k in range(0, len(MP_Profile)):
    plt.plot(Coordinates[:], Temperature[:], color=colors[k], label=labels[k])
    
plt.xlabel(r'Position in Supercell ($\AA$)')
plt.ylabel('Temperature (K)')
plt.title('Muller-Plathe Temperature Gradient', fontsize = 24)

# Load the data that will be averaged to calculate the slopes 
Temperature = []; Coordinate = []; dX_Thermo = [];

for simNo in range(0, len(filename)):
    
    dX_ = Thermo_Data[simNo,-1,9]
    dX_Thermo.append(dX_)
    
    for i in range(0,800):
        Temperature_ = np.mean([MP_Ave[simNo,i,3],MP_Ave[simNo,i+800,3],MP_Ave[simNo,i+1600,3],MP_Ave[simNo,i+2400,3]])   
        Temperature.append(Temperature_)
        Coordinate_ = np.mean([MP_Ave[simNo,i,1],MP_Ave[simNo,i+800,1],MP_Ave[simNo,i+1600,1],MP_Ave[simNo,i+2400,1]])*dX_
        Coordinate.append(Coordinate_)
    
Temperature = np.array([Temperature[0:800],Temperature[800:1600],Temperature[1600:2400],
                       Temperature[2400:3200],Temperature[3200:4000]], dtype=float) 
                       
Coordinate = np.array([Coordinate[0:800],Coordinate[800:1600],Coordinate[1600:2400],
                      Coordinate[2400:3200],Coordinate[3200:4000]], dtype=float) 
                      

excluded_points1 = 230 # excluded points from the beginning
excluded_points2 = 40 # excluded points from the end
excluded_points3 = 20
excluded_points4 = 230 # from the end of the mp.data file

slope2_mean = []; slope2 = []; slope1_mean = []; slope1 = []; average_temp_Prst = []

for simNo in range(0,len(filename)):
    
    dTemp = Temperature[simNo,:]
    X = dX_
    dX = Coordinate[simNo,:] #*X # normalized distance
        
    T_fit_left = np.polyfit(dX[excluded_points1:400-excluded_points2], dTemp[excluded_points1:400-excluded_points2], 1)
    T_fit_right = -np.polyfit(dX[400+excluded_points2:-excluded_points1], dTemp[400+excluded_points2:-excluded_points1], 1)   
    slope2.append([T_fit_left[0], T_fit_right[0]])
    mean_slope = np.mean([abs(T_fit_left[0]), abs(T_fit_right[0])], axis = 0)
    slope2_mean.append(mean_slope)
    B_fit_left2 = np.polyfit(dX[excluded_points3:400-excluded_points4], dTemp[excluded_points3:400-excluded_points4], 1)
    B_fit_right2 = -np.polyfit(dX[400+excluded_points4:-excluded_points3], dTemp[400+excluded_points4:-excluded_points3], 1)
    slope1.append([B_fit_left2[0], B_fit_right2[0]])    
    mean_slope = np.mean([abs(B_fit_left2[0]), abs(B_fit_right2[0])], axis = 0)
    slope1_mean.append(mean_slope)
    average_temp = np.mean(dTemp)
    average_temp_Prst.append(average_temp)
    
    
Slope1 = np.array(slope1)
Slope_mean1 = np.array(slope1_mean)

Slope2 = np.array(slope2)
Slope_mean2 = np.array(slope2_mean)

Slope_mean = np.array([np.mean([Slope_mean1[0], Slope_mean2[0]], axis=0)]) #individual
#Slope_mean = np.array([np.mean([Slope_mean1[0], Slope_mean2[0]], axis=0), np.mean([Slope_mean1[1], Slope_mean2[1]], axis =0), np.mean([Slope_mean1[2], Slope_mean2[2]], axis=0)])

Slope_ave = np.array([Slope_mean[0]*dX_Thermo[0]]) #individual
#Slope_ave = np.array([Slope_mean[0]*dX_Thermo[0], Slope_mean[1]*dX_Thermo[1], Slope_mean[2]*dX_Thermo[2]])

# Calculate the heat flux and the thermal conductivity
eV2J  = 1.6e-19;
Ang2m =   1e-10;
ps2s  =   1e-12;
conversion = (eV2J/(Ang2m*ps2s))

s_values = []
ThermalsPrst = []
dt = 0.0005 # picoseconds

for simNo in range(0, len(Thermo_Data)):
    
    Steps = Thermo_Data[simNo,-1,0]
    Time = Steps*dt # picoseconds
    
    dx = Thermo_Data[simNo,-1,9]
    dy = Thermo_Data[simNo,-1,10]
    dz = Thermo_Data[simNo,-1,11]
    Area = dy*dz
    
    #DeltaE = Thermo_Data[simNo,-1,-2] # E_pair
    #TotalE = Thermo_Data[simNo,-1,-5] # Total energy
    KinE = Thermo_Data[simNo,-1,12] # in log_lammps, f_3 
    
    Thermal_left2 = ((KinE)/(2*Time*Area*Slope2[simNo,0]))*conversion #top left
    Thermal_right2 = ((KinE)/(2*Time*Area*Slope2[simNo,1]))*conversion #top right
    
    Thermal_left1 = ((KinE)/(2*Time*Area*Slope1[simNo,0]))*conversion #bottom left
    Thermal_right1 = ((KinE)/(2*Time*Area*Slope1[simNo,1]))*conversion #bottom right
    
    thermal2 = np.concatenate((Thermal_left1, Thermal_left2), axis = None)
    thermal1 = np.concatenate((Thermal_right1, Thermal_right2), axis = None)
    
    thermal2_ave = ((KinE)/(2*Time*Area*Slope_mean2[simNo]))*conversion #top
    thermal1_ave = ((KinE)/(2*Time*Area*Slope_mean1[simNo]))*conversion #bottom
    
    #thermal_ave = ((KinE)/(2*Time*Area*Slope_ave[simNo]))*conversion
    thermal_ave = (thermal2_ave+thermal1_ave)/2
   
    getcontext().prec = 10
    std_top = Decimal(np.std([Thermal_left2, Thermal_right2], dtype=np.float64))
    SE_left = std_top/Decimal(math.sqrt(2))
   
    std_bottom = Decimal(np.std([Thermal_left1, Thermal_right1], dtype=np.float64))
    SE_right = std_bottom/Decimal(math.sqrt(2))
    
    SE = math.sqrt(((SE_left/2)**2) + ((SE_right/2)**2))
    
    Therm_val = [Thermal_left1,Thermal_left2,Thermal_right1,Thermal_right2]
    Ttherm = ['%.6f' % elem for elem in Therm_val]
    ThermalsPrst.append(Ttherm)
    
    
    TC_values = [thermal2_ave, thermal1_ave, thermal_ave, SE]
    val = ['%.6f' % elem for elem in TC_values]
    s_values.append(val)
    

therm_average_prst = []
for item in s_values:
    therm_average_prst.append(item[2])

therm_average_prst = []
for item in s_values:
    therm_average_prst.append(np.array(float(item[2])))

#projection stuff
size_prst = [200.0,400.0,600.0,800.0,1000.0]

size_prst = np.array(size_prst)
Tc = [float(s_values[0][2]),float(s_values[1][2]),float(s_values[2][2]),float(s_values[3][2]),float(s_values[4][2])]#,
Tc = np.array(Tc)
inv_size_prst = np.reciprocal(size_prst)
inv_Tc = np.reciprocal(Tc)

#plot
fig = plt.figure(figsize=(12,8))

plt.plot(size_prst[:], Tc[:],'o-')


plt.xlabel(r'Length', fontdict={'fontsize':27})
plt.ylabel(r'k', fontdict={'fontsize':27})
plt.title(r'Projection of Convergence for Pristine Si', fontdict={'fontsize':27})

fig = plt.figure(figsize=(12,8))

x1 = np.array(size_prst)
mn = min(x1)
mx = max(x1)
y1 = np.array(Tc)

z1 = np.polyfit(x1, y1, 1) #np.polyfit(x, y, degree of polynomial), degree =1,2,3 is linear, quadratic, cubic
z2 = np.polyfit(x1, y1, 2)
z3 = np.polyfit(x1, y1, 3)

xp = np.linspace(-0.001, mx+0.001, 6)

p1 = np.poly1d(z1)
p2 = np.poly1d(z2)
p3 = np.poly1d(z3)
#pz = np.poly1d(np.polyfit(x, y, 3)) #same as p3

plt.plot(x1, y1, 'o-', color ='black', label ='OG')
plt.plot(xp, p1(xp), 's', label = 'linear')
plt.plot(xp, p2(xp), 's', label = 'quadratic')
plt.plot(xp, p3(xp), 's', label ='cubic') 

plt.axvline(x=125.0, color=color_blind_safe[3])

plt.legend(loc="lower right")

plt.xlim([0.0,2000])
plt.ylim([0.0,20])

plt.xlabel(r'Length', fontdict={'fontsize':27})
plt.ylabel(r'k', fontdict={'fontsize':27})
plt.title(r'Projection of Convergence for Pristine Si', fontdict={'fontsize':27})

fig = plt.figure(figsize=(12,8))

x_2 = np.array(size_prst)
mn = min(x_2)
mx = max(x_2)

y_2 = np.array(Tc)

xpp = np.linspace(-0.0005,mx, 6)

plt.axvline(x=125.0, color=color_blind_safe[3])


z3 = np.polyfit(x_2, y_2, 3)
p3 = np.poly1d(z3)
#pz = np.poly1d(np.polyfit(x, y, 3)) #same as p3

s = p3(1);
i = p3(2)    #The polyfit function for a linear (polynomial order 1) fit returns the slope as the first parameter
                    #and the intercept as the second parameter, so the output vector is [slope, intercept]. 
                    #It is like any other vector, so choose the one you want by indexing into it.
n = p3(0)
slope_intercept = np.polyfit(x_2,y_2,3)

plt.plot(x_2,y_2, 'o-', label ='OG')
plt.plot(xpp, p3(xp), 's', label ='cubic') 
plt.legend(loc="lower right")

plt.xlabel(r'Length', fontdict={'fontsize':27})
plt.ylabel(r'k', fontdict={'fontsize':27})
plt.title(r'Projection of Convergence for Pristine Si cubic', fontdict={'fontsize':27})

#linear fit in linear region alone
fig = plt.figure(figsize=(12,8))

x2 = size_prst
x2 = np.array(x2)
y2 = Tc
y2 = np.array(y2)


z = np.polyfit(x2, y2, 1) 
xp = np.linspace(-0.001, mx+0.001, 3)

p1 = np.poly1d(z)

slope_intercept = np.polyfit(x2,y2,1)


plt.plot(x2, y2, 'o-', color ='black', label ='OG')
plt.plot(xp, p1(xp), 's', label = 'linear')

plt.axvline(x=0, color=color_blind_safe[3])

plt.legend(loc="lower right")

plt.xlabel(r'Length', fontdict={'fontsize':27})
plt.ylabel(r'k', fontdict={'fontsize':27})
plt.title(r'Projection of Convergence for linear Pristine Si', fontdict={'fontsize':27})



#Pristine Si values

SE200b_left = np.std([float(ThermalsPrst[0][0]), float(ThermalsPrst[0][2])], dtype=np.float64)
SE200_left = SE200b_left/math.sqrt(2)
SE200b_right = np.std([float(ThermalsPrst[0][1]), float(ThermalsPrst[0][3])], dtype=np.float64)
SE200_right = SE200b_right/math.sqrt(2)
standard_error200b_prst = math.sqrt(((SE200_left/2)**2) + ((SE200_right/2)**2))

SE400b_left = np.std([float(ThermalsPrst[1][0]), float(ThermalsPrst[1][2])], dtype=np.float64)
SE400_left = SE400b_left/math.sqrt(2)
SE400b_right = np.std([float(ThermalsPrst[1][1]), float(ThermalsPrst[1][3])], dtype=np.float64)
SE400_right = SE400b_right/math.sqrt(2)
standard_error400b_prst = math.sqrt(((SE400_left/2)**2) + ((SE400_right/2)**2))


SE600b_left = np.std([float(ThermalsPrst[2][0]), float(ThermalsPrst[2][2])], dtype=np.float64)
SE600_left = SE600b_left/math.sqrt(2)
SE600b_right = np.std([float(ThermalsPrst[2][1]), float(ThermalsPrst[2][3])], dtype=np.float64)
SE600_right = SE600b_right/math.sqrt(2)
standard_error600b_prst = math.sqrt(((SE600_left/2)**2) + ((SE600_right/2)**2))

SE800b_left = np.std([float(ThermalsPrst[3][0]), float(ThermalsPrst[3][2])], dtype=np.float64)
SE800_left = SE800b_left/math.sqrt(2)
SE800b_right = np.std([float(ThermalsPrst[3][1]), float(ThermalsPrst[3][3])], dtype=np.float64)
SE800_right = SE800b_right/math.sqrt(2)
standard_error800b_prst = math.sqrt(((SE800_left/2)**2) + ((SE800_right/2)**2))

SE1000b_left = np.std([float(ThermalsPrst[4][0]), float(ThermalsPrst[4][2])], dtype=np.float64)
SE1000_left = SE1000b_left/math.sqrt(2)
SE1000b_right = np.std([float(ThermalsPrst[4][1]), float(ThermalsPrst[4][3])], dtype=np.float64)
SE1000_right = SE1000b_right/math.sqrt(2)
standard_error1000b_prst = math.sqrt(((SE1000_left/2)**2) + ((SE1000_right/2)**2))

yerr200b_prst = standard_error200b_prst
yerr400b_prst = standard_error400b_prst
yerr600b_prst = standard_error600b_prst
yerr800b_prst = standard_error800b_prst
yerr1000b_prst = standard_error1000b_prst


In [ ]:
filename = ['./profile_str_siE200','profile_str_siE400', 'profile_str_siE600',
            'profile_str_siE800', 'profile_str_siE1000']
           


MP_Profile = []; MP_Ave = []; MP_LastN = []; 

for simNo in range(0,len(filename)):
    subprocess.call(r"perl -p -e 's/^[\ \t]+|[\ \t]+$//g' %s > %s_new" % (filename[simNo], filename[simNo]), shell=True) # remove trailling white spaces
    subprocess.call("mv %s_new %s" % (filename[simNo], filename[simNo]), shell=True)
    subprocess.call("tr -s ' ' < %s > %s_tmp " % (filename[simNo], filename[simNo]), shell=True) #squeeze spaces
    subprocess.call("mv %s_tmp %s" % (filename[simNo], filename[simNo]), shell=True)

    profile = []; N = 800;
    with open(filename[simNo], 'r') as f:
        while True:
            lines = list(itertools.islice(f, 1, N+1))
            for line in lines:
                profile.append(line.split(' '))
            if not lines: 
                break
                
    lastN = profile[-N:] 
    lastAve = profile[-3200:]

    MP_Profile.append(profile)
    MP_LastN.append(lastN)
    MP_Ave.append(lastAve)
    
MP_Profile = np.array(MP_Profile, dtype=float) # for plotting 3D graphs

MP_LastN = np.array(MP_LastN, dtype=float) # for calculating the thermal conductivity



MP_Ave = np.array(MP_Ave, dtype=float) # for averaging the data to calculate the thermal conductivity

Thermo_Data = np.array([np.loadtxt('./Thermo_str_siE200'),np.loadtxt('./Thermo_str_siE400'),np.loadtxt('./Thermo_str_siE600'),
                       np.loadtxt('./Thermo_str_siE800'),np.loadtxt('./Thermo_str_siE1000')], dtype = object) 
                        
labels = ['str_si200', 'str_si600', 'str_si600','str_si800','str_si1000','str_si1500','str_si2000'] 

for simNo in range(0, len(MP_Profile)):
    Coordinates = MP_Profile[simNo,:,1]*(Thermo_Data[simNo,-1,9])
    Temperature = MP_Profile[simNo,:,3]

fig = plt.figure(figsize=(12,8))
for k in range(0, len(MP_Profile)):
    plt.plot(Coordinates[:], Temperature[:], color=colors[k], label=labels[k])
    
plt.xlabel(r'Position in Supercell ($\AA$)')
plt.ylabel('Temperature (K)')
plt.title('Muller-Plathe Temperature Gradient', fontsize = 24)

# Load the data that will be averaged to calculate the slopes 
Temperature = []; Coordinate = []; dX_Thermo = [];

for simNo in range(0, len(filename)):
    
    dX_ = Thermo_Data[simNo,-1,9]
    dX_Thermo.append(dX_)
    
    for i in range(0,800):
        Temperature_ = np.mean([MP_Ave[simNo,i,3],MP_Ave[simNo,i+800,3],MP_Ave[simNo,i+1600,3],MP_Ave[simNo,i+2400,3]])   
        Temperature.append(Temperature_)
        Coordinate_ = np.mean([MP_Ave[simNo,i,1],MP_Ave[simNo,i+800,1],MP_Ave[simNo,i+1600,1],MP_Ave[simNo,i+2400,1]])*dX_
        Coordinate.append(Coordinate_)
    
Temperature = np.array([Temperature[0:800],Temperature[800:1600],Temperature[1600:2400],
                       Temperature[2400:3200],Temperature[3200:4000]], dtype=float) 
                       
Coordinate = np.array([Coordinate[0:800],Coordinate[800:1600],Coordinate[1600:2400],
                      Coordinate[2400:3200],Coordinate[3200:4000]], dtype=float) 
                      
excluded_points1 = 200 # excluded points from the beginning
excluded_points2 = 40 # excluded points from the end
excluded_points3 = 20
excluded_points4 = 230 # from the end of the mp.data file

slope2_mean = []; slope2 = []; slope1_mean = []; slope1 = []; average_temp_str_SiGeSi = [];

# Step Temp E_pair PotEng KinEng TotEng Temp Press Volume Lx Ly Lz f_3 v_tdiff
for simNo in range(0,len(filename)):
    
    dTemp = Temperature[simNo,:]
    X = dX_
    dX = Coordinate[simNo,:] #*X # normalized distance
    T_fit_left = np.polyfit(dX[excluded_points1:400-excluded_points2], dTemp[excluded_points1:400-excluded_points2], 1)
    T_fit_right = -np.polyfit(dX[400+excluded_points2:-excluded_points1], dTemp[400+excluded_points2:-excluded_points1], 1)
    slope2.append([T_fit_left[0], T_fit_right[0]])
    mean_slope = np.mean([abs(T_fit_left[0]), abs(T_fit_right[0])], axis = 0)
    slope2_mean.append(mean_slope)
    B_fit_left2 = np.polyfit(dX[excluded_points3:400-excluded_points4], dTemp[excluded_points3:400-excluded_points4], 1)
    B_fit_right2 = -np.polyfit(dX[400+excluded_points4:-excluded_points3], dTemp[400+excluded_points4:-excluded_points3], 1)
    slope1.append([B_fit_left2[0], B_fit_right2[0]])
    mean_slope = np.mean([abs(B_fit_left2[0]), abs(B_fit_right2[0])], axis = 0)
    slope1_mean.append(mean_slope)
    average_temp = np.mean(dTemp)
    average_temp_str_SiGeSi.append(average_temp)    

Slope1 = np.array(slope1)
Slope_mean1 = np.array(slope1_mean)

Slope2 = np.array(slope2)
Slope_mean2 = np.array(slope2_mean)

Slope_mean = np.array([np.mean([Slope_mean1[0], Slope_mean2[0]], axis=0)]) #individual
#Slope_mean = np.array([np.mean([Slope_mean1[0], Slope_mean2[0]], axis=0), np.mean([Slope_mean1[1], Slope_mean2[1]], axis =0), np.mean([Slope_mean1[2], Slope_mean2[2]], axis=0)])

Slope_ave = np.array([Slope_mean[0]*dX_Thermo[0]]) #individual
#Slope_ave = np.array([Slope_mean[0]*dX_Thermo[0], Slope_mean[1]*dX_Thermo[1], Slope_mean[2]*dX_Thermo[2]])

# Calculate the heat flux and the thermal conductivity
eV2J  = 1.6e-19;
Ang2m =   1e-10;
ps2s  =   1e-12;
conversion = (eV2J/(Ang2m*ps2s))

s_values = []
Thermals_str_SiGeSi = []
dt = 0.0005 # picoseconds

# Step Temp E_pair PotEng KinEng TotEng Temp Press Volume Lx Ly Lz f_3 v_tdiff
for simNo in range(0, len(Thermo_Data)):
    
    Steps = Thermo_Data[simNo,-1,0]
    Time = Steps*dt # picoseconds
    
    dx = Thermo_Data[simNo,-1,9]
    dy = Thermo_Data[simNo,-1,10]
    dz = Thermo_Data[simNo,-1,11]
    Area = dy*dz
    
    #DeltaE = Thermo_Data[simNo,-1,-2] # E_pair
    #TotalE = Thermo_Data[simNo,-1,-5] # Total energy
    KinE = Thermo_Data[simNo,-1,12] # in log_lammps, f_3 
    
    Thermal_left2 = ((KinE)/(2*Time*Area*Slope2[simNo,0]))*conversion #top left
    Thermal_right2 = ((KinE)/(2*Time*Area*Slope2[simNo,1]))*conversion #top right
    
    Thermal_left1 = ((KinE)/(2*Time*Area*Slope1[simNo,0]))*conversion #bottom left
    Thermal_right1 = ((KinE)/(2*Time*Area*Slope1[simNo,1]))*conversion #bottom right
    
    thermal2 = np.concatenate((Thermal_left1, Thermal_left2), axis = None)
    thermal1 = np.concatenate((Thermal_right1, Thermal_right2), axis = None)
    
    thermal2_ave = ((KinE)/(2*Time*Area*Slope_mean2[simNo]))*conversion #top
    thermal1_ave = ((KinE)/(2*Time*Area*Slope_mean1[simNo]))*conversion #bottom
    
    #thermal_ave = ((KinE)/(2*Time*Area*Slope_ave[simNo]))*conversion
    thermal_ave = (thermal2_ave+thermal1_ave)/2
   
    getcontext().prec = 10
    std_top = Decimal(np.std([Thermal_left2, Thermal_right2], dtype=np.float64))
    SE_left = std_top/Decimal(math.sqrt(2))
   
    std_bottom = Decimal(np.std([Thermal_left1, Thermal_right1], dtype=np.float64))
    SE_right = std_bottom/Decimal(math.sqrt(2))
    
    SE = math.sqrt(((SE_left/2)**2) + ((SE_right/2)**2))
    
    Therm_val = [Thermal_left1,Thermal_left2,Thermal_right1,Thermal_right2]
    Ttherm = ['%.2f' % elem for elem in Therm_val]
    Thermals_str_SiGeSi.append(Ttherm)
    
    
    TC_values = [thermal2_ave, thermal1_ave, thermal_ave, SE]
    val = ['%.2f' % elem for elem in TC_values]
    s_values.append(val)



therm_average_str_SiGeSi = []
for item in s_values:
    therm_average_str_SiGeSi.append(item[2])

therm_average_str_SiGeSi = []
for item in s_values:
    therm_average_str_SiGeSi.append(np.array(float(item[2])))

#projection stuff
str_SiGeSi = [200.0,400.0,600.0,800.0,1000.0]#,1500.0,2000.0]
str_SiGeSi = np.array(str_SiGeSi)
Tc = [float(s_values[0][2]),float(s_values[1][2]),float(s_values[2][2]),
      float(s_values[3][2]),float(s_values[4][2])]
Tc = np.array(Tc)

inv_str_SiGeSi = np.reciprocal(str_SiGeSi)

inv_Tc = np.reciprocal(Tc)

#plot
fig = plt.figure(figsize=(12,8))

plt.plot(str_SiGeSi[:], Tc[:],'o-')


plt.xlabel(r'Length', fontdict={'fontsize':27})
plt.ylabel(r'k', fontdict={'fontsize':27})
plt.title(r'Projection of Convergence for Straight SiGeSi', fontdict={'fontsize':27})



fig = plt.figure(figsize=(12,8))

x1 = np.array(str_SiGeSi)
mn = min(x1)
mx = max(x1)

y1 = np.array(Tc)

z1 = np.polyfit(x1, y1, 1) #np.polyfit(x, y, degree of polynomial), degree =1,2,3 is linear, quadratic, cubic
z2 = np.polyfit(x1, y1, 2)
z3 = np.polyfit(x1, y1, 3)

xp = np.linspace(-0.001, mx+0.001, 6)

p1 = np.poly1d(z1)
p2 = np.poly1d(z2)
p3 = np.poly1d(z3)
#pz = np.poly1d(np.polyfit(x, y, 3)) #same as p3

plt.plot(x1, y1, 'o-', color ='black', label ='OG')
plt.plot(xp, p1(xp), 's', label = 'linear')
plt.plot(xp, p2(xp), 's', label = 'quadratic')
plt.plot(xp, p3(xp), 's', label ='cubic') 

plt.axvline(x=125.0, color=color_blind_safe[3])

plt.legend(loc="lower right")

plt.xlim([0.0,2000])
plt.ylim([0.0,20])

plt.xlabel(r'Length', fontdict={'fontsize':27})
plt.ylabel(r'k', fontdict={'fontsize':27})
plt.title(r'Projection of Convergence for Straight SiGeSi', fontdict={'fontsize':27})

fig = plt.figure(figsize=(12,8))

x_2 = np.array(str_SiGeSi)
mn = min(x_2)
mx = max(x_2)

y_2 = np.array(Tc)
xpp = np.linspace(-0.0005,mx, 6)

plt.axvline(x=125.0, color=color_blind_safe[3])


z3 = np.polyfit(x_2, y_2, 3)
p3 = np.poly1d(z3)
#pz = np.poly1d(np.polyfit(x, y, 3)) #same as p3

s = p3(1);
i = p3(2)    #The polyfit function for a linear (polynomial order 1) fit returns the slope as the first parameter
                    #and the intercept as the second parameter, so the output vector is [slope, intercept]. 
                    #It is like any other vector, so choose the one you want by indexing into it.
n = p3(0)
slope_intercept = np.polyfit(x_2,y_2,3)

plt.plot(x_2,y_2, 'o-', label ='OG')
plt.plot(xpp, p3(xp), 's', label ='cubic') 
plt.legend(loc="lower right")

plt.xlabel(r'Length', fontdict={'fontsize':27})
plt.ylabel(r'k', fontdict={'fontsize':27})
plt.title(r'Projection of Convergence for Straight SiGeSi cubic', fontdict={'fontsize':27})

#linear fit in linear region alone
fig = plt.figure(figsize=(12,8))

x2 = str_SiGeSi
x2 = np.array(x2)
y2 = Tc
y2 = np.array(y2)


z = np.polyfit(x2, y2, 1) 
xp = np.linspace(-0.001, mx+0.001, 3)

p1 = np.poly1d(z)

slope_intercept = np.polyfit(x2,y2,1)

plt.plot(x2, y2, 'o-', color ='black', label ='OG')
plt.plot(xp, p1(xp), 's', label = 'linear')

plt.axvline(x=0, color=color_blind_safe[3])

plt.legend(loc="lower right")

plt.xlabel(r'Length', fontdict={'fontsize':27})
plt.ylabel(r'k', fontdict={'fontsize':27})
plt.title(r'Projection of Convergence for linear Straight SiGeSi', fontdict={'fontsize':27})



SE200b_left = np.std([float(Thermals_str_SiGeSi[0][0]), float(Thermals_str_SiGeSi[0][2])], dtype=np.float64)
SE200_left = SE200b_left/math.sqrt(2)
SE200b_right = np.std([float(Thermals_str_SiGeSi[0][1]), float(Thermals_str_SiGeSi[0][3])], dtype=np.float64)
SE200_right = SE200b_right/math.sqrt(2)
standard_error200b_str_SiGeSi = math.sqrt(((SE200_left/2)**2) + ((SE200_right/2)**2))

SE400b_left = np.std([float(Thermals_str_SiGeSi[1][0]), float(Thermals_str_SiGeSi[1][2])], dtype=np.float64)
SE400_left = SE400b_left/math.sqrt(2)
SE400b_right = np.std([float(Thermals_str_SiGeSi[1][1]), float(Thermals_str_SiGeSi[1][3])], dtype=np.float64)
SE400_right = SE400b_right/math.sqrt(2)
standard_error400b_str_SiGeSi = math.sqrt(((SE400_left/2)**2) + ((SE400_right/2)**2))

SE600b_left = np.std([float(Thermals_str_SiGeSi[2][0]), float(Thermals_str_SiGeSi[2][2])], dtype=np.float64)
SE600_left = SE600b_left/math.sqrt(2)
SE600b_right = np.std([float(Thermals_str_SiGeSi[2][1]), float(Thermals_str_SiGeSi[2][3])], dtype=np.float64)
SE600_right = SE600b_right/math.sqrt(2)
standard_error600b_str_SiGeSi = math.sqrt(((SE600_left/2)**2) + ((SE600_right/2)**2))

SE800b_left = np.std([float(Thermals_str_SiGeSi[3][0]), float(Thermals_str_SiGeSi[3][2])], dtype=np.float64)
SE800_left = SE800b_left/math.sqrt(2)
SE800b_right = np.std([float(Thermals_str_SiGeSi[3][1]), float(Thermals_str_SiGeSi[3][3])], dtype=np.float64)
SE800_right = SE800b_right/math.sqrt(2)
standard_error800b_str_SiGeSi = math.sqrt(((SE800_left/2)**2) + ((SE800_right/2)**2))

SE1000b_left = np.std([float(Thermals_str_SiGeSi[4][0]), float(Thermals_str_SiGeSi[4][2])], dtype=np.float64)
SE1000_left = SE1000b_left/math.sqrt(2)
SE1000b_right = np.std([float(Thermals_str_SiGeSi[4][1]), float(Thermals_str_SiGeSi[4][3])], dtype=np.float64)
SE1000_right = SE1000b_right/math.sqrt(2)
standard_error1000b_str_SiGeSi = math.sqrt(((SE1000_left/2)**2) + ((SE1000_right/2)**2))

yerr200b_str_SiGeSi = standard_error200b_str_SiGeSi
yerr400b_str_SiGeSi = standard_error400b_str_SiGeSi
yerr600b_str_SiGeSi = standard_error600b_str_SiGeSi
yerr800b_str_SiGeSi = standard_error800b_str_SiGeSi
yerr1000b_str_SiGeSi = standard_error1000b_str_SiGeSi


In [ ]:
filename = ['./profile_str_geE200','profile_str_geE400', 'profile_str_geE600',
            'profile_str_geE800', 'profile_str_geE1000']
           


MP_Profile = []; MP_Ave = []; MP_LastN = []; 

for simNo in range(0,len(filename)):
    subprocess.call(r"perl -p -e 's/^[\ \t]+|[\ \t]+$//g' %s > %s_new" % (filename[simNo], filename[simNo]), shell=True) # remove trailling white spaces
    subprocess.call("mv %s_new %s" % (filename[simNo], filename[simNo]), shell=True)
    subprocess.call("tr -s ' ' < %s > %s_tmp " % (filename[simNo], filename[simNo]), shell=True) #squeeze spaces
    subprocess.call("mv %s_tmp %s" % (filename[simNo], filename[simNo]), shell=True)

    profile = []; N = 800;
    with open(filename[simNo], 'r') as f:
        while True:
            lines = list(itertools.islice(f, 1, N+1))
            for line in lines:
                profile.append(line.split(' '))
            if not lines: 
                break
                
    lastN = profile[-N:] 
    lastAve = profile[-3200:]

    MP_Profile.append(profile)
    MP_LastN.append(lastN)
    MP_Ave.append(lastAve)
    
MP_Profile = np.array(MP_Profile, dtype=float) # for plotting 3D graphs
MP_LastN = np.array(MP_LastN, dtype=float) # for calculating the thermal conductivity


MP_Ave = np.array(MP_Ave, dtype=float) # for averaging the data to calculate the thermal conductivity

Thermo_Data = np.array([np.loadtxt('./Thermo_str_geE200'),np.loadtxt('./Thermo_str_geE400'),np.loadtxt('./Thermo_str_geE600'),
                       np.loadtxt('./Thermo_str_geE800'),np.loadtxt('./Thermo_str_geE1000')], dtype = object) 
                        
labels = ['str_ge200', 'str_ge600', 'str_ge600','str_ge800','str_ge1000','str_ge1500','str_ge2000'] 


for simNo in range(0, len(MP_Profile)):
    Coordinates = MP_Profile[simNo,:,1]*(Thermo_Data[simNo,-1,9])
    Temperature = MP_Profile[simNo,:,3]

fig = plt.figure(figsize=(12,8))
for k in range(0, len(MP_Profile)):
    plt.plot(Coordinates[:], Temperature[:], color=colors[k], label=labels[k])
    
plt.xlabel(r'Position in Supercell ($\AA$)')
plt.ylabel('Temperature (K)')
plt.title('Muller-Plathe Temperature Gradient', fontsize = 24)

# Load the data that will be averaged to calculate the slopes 
Temperature = []; Coordinate = []; dX_Thermo = [];

for simNo in range(0, len(filename)):
    
    dX_ = Thermo_Data[simNo,-1,9]
    dX_Thermo.append(dX_)
    
    for i in range(0,800):
        Temperature_ = np.mean([MP_Ave[simNo,i,3],MP_Ave[simNo,i+800,3],MP_Ave[simNo,i+1600,3],MP_Ave[simNo,i+2400,3]])   
        Temperature.append(Temperature_)
        Coordinate_ = np.mean([MP_Ave[simNo,i,1],MP_Ave[simNo,i+800,1],MP_Ave[simNo,i+1600,1],MP_Ave[simNo,i+2400,1]])*dX_
        Coordinate.append(Coordinate_)
    
Temperature = np.array([Temperature[0:800],Temperature[800:1600],Temperature[1600:2400],
                       Temperature[2400:3200],Temperature[3200:4000]], dtype=float) 
                      
Coordinate = np.array([Coordinate[0:800],Coordinate[800:1600],Coordinate[1600:2400],
                      Coordinate[2400:3200],Coordinate[3200:4000]], dtype=float) 
                      


excluded_points1 = 200 # excluded points from the beginning
excluded_points2 = 40 # excluded points from the end
excluded_points3 = 20
excluded_points4 = 230 # from the end of the mp.data file

slope2_mean = []; slope2 = []; slope1_mean = []; slope1 = []; average_temp_str_GeSiGe = [];

for simNo in range(0,len(filename)):
    
    dTemp = Temperature[simNo,:]
    X = dX_
    dX = Coordinate[simNo,:] #*X # normalized distance
        
    T_fit_left = np.polyfit(dX[excluded_points1:400-excluded_points2], dTemp[excluded_points1:400-excluded_points2], 1)
    T_fit_right = -np.polyfit(dX[400+excluded_points2:-excluded_points1], dTemp[400+excluded_points2:-excluded_points1], 1)
    
    slope2.append([T_fit_left[0], T_fit_right[0]])

    mean_slope = np.mean([abs(T_fit_left[0]), abs(T_fit_right[0])], axis = 0)
    slope2_mean.append(mean_slope)
    
    B_fit_left2 = np.polyfit(dX[excluded_points3:400-excluded_points4], dTemp[excluded_points3:400-excluded_points4], 1)
    B_fit_right2 = -np.polyfit(dX[400+excluded_points4:-excluded_points3], dTemp[400+excluded_points4:-excluded_points3], 1)
    
    slope1.append([B_fit_left2[0], B_fit_right2[0]])
    
    mean_slope = np.mean([abs(B_fit_left2[0]), abs(B_fit_right2[0])], axis = 0)
    slope1_mean.append(mean_slope)
 
    average_temp = np.mean(dTemp)
    average_temp_str_GeSiGe.append(average_temp)    

Slope1 = np.array(slope1)
Slope_mean1 = np.array(slope1_mean)

Slope2 = np.array(slope2)
Slope_mean2 = np.array(slope2_mean)

Slope_mean = np.array([np.mean([Slope_mean1[0], Slope_mean2[0]], axis=0)]) #individual
#Slope_mean = np.array([np.mean([Slope_mean1[0], Slope_mean2[0]], axis=0), np.mean([Slope_mean1[1], Slope_mean2[1]], axis =0), np.mean([Slope_mean1[2], Slope_mean2[2]], axis=0)])

Slope_ave = np.array([Slope_mean[0]*dX_Thermo[0]]) #individual
#Slope_ave = np.array([Slope_mean[0]*dX_Thermo[0], Slope_mean[1]*dX_Thermo[1], Slope_mean[2]*dX_Thermo[2]])

# Calculate the heat flux and the thermal conductivity
eV2J  = 1.6e-19;
Ang2m =   1e-10;
ps2s  =   1e-12;
conversion = (eV2J/(Ang2m*ps2s))

s_values = []
Thermals_str_GeSiGe = []
dt = 0.0005 # picoseconds

for simNo in range(0, len(Thermo_Data)):
    
    Steps = Thermo_Data[simNo,-1,0]
    Time = Steps*dt # picoseconds
    
    dx = Thermo_Data[simNo,-1,9]
    dy = Thermo_Data[simNo,-1,10]
    dz = Thermo_Data[simNo,-1,11]
    Area = dy*dz
    
    #DeltaE = Thermo_Data[simNo,-1,-2] # E_pair
    #TotalE = Thermo_Data[simNo,-1,-5] # Total energy
    KinE = Thermo_Data[simNo,-1,12] # in log_lammps, f_3 
    
    Thermal_left2 = ((KinE)/(2*Time*Area*Slope2[simNo,0]))*conversion #top left
    Thermal_right2 = ((KinE)/(2*Time*Area*Slope2[simNo,1]))*conversion #top right
    
    Thermal_left1 = ((KinE)/(2*Time*Area*Slope1[simNo,0]))*conversion #bottom left
    Thermal_right1 = ((KinE)/(2*Time*Area*Slope1[simNo,1]))*conversion #bottom right
    
    thermal2 = np.concatenate((Thermal_left1, Thermal_left2), axis = None)
    thermal1 = np.concatenate((Thermal_right1, Thermal_right2), axis = None)
    
    thermal2_ave = ((KinE)/(2*Time*Area*Slope_mean2[simNo]))*conversion #top
    thermal1_ave = ((KinE)/(2*Time*Area*Slope_mean1[simNo]))*conversion #bottom
    
    #thermal_ave = ((KinE)/(2*Time*Area*Slope_ave[simNo]))*conversion
    thermal_ave = (thermal2_ave+thermal1_ave)/2
   
    getcontext().prec = 10
    std_top = Decimal(np.std([Thermal_left2, Thermal_right2], dtype=np.float64))
    SE_left = std_top/Decimal(math.sqrt(2))
   
    std_bottom = Decimal(np.std([Thermal_left1, Thermal_right1], dtype=np.float64))
    SE_right = std_bottom/Decimal(math.sqrt(2))
    
    SE = math.sqrt(((SE_left/2)**2) + ((SE_right/2)**2))
    
    Therm_val = [Thermal_left1,Thermal_left2,Thermal_right1,Thermal_right2]
    Ttherm = ['%.2f' % elem for elem in Therm_val]
    Thermals_str_GeSiGe.append(Ttherm)
    
    
    TC_values = [thermal2_ave, thermal1_ave, thermal_ave, SE]
    val = ['%.2f' % elem for elem in TC_values]
    s_values.append(val)

therm_average_str_GeSiGe = []
for item in s_values:
    therm_average_str_GeSiGe.append(item[2])

therm_average_str_GeSiGe = []
for item in s_values:
    therm_average_str_GeSiGe.append(np.array(float(item[2])))

#projection stuff
str_GeSiGe = [200.0,400.0,600.0,800.0,1000.0]#,1500.0,2000.0]
str_GeSiGe = np.array(str_GeSiGe)
Tc = [float(s_values[0][2]),float(s_values[1][2]),float(s_values[2][2]),
      float(s_values[3][2]),float(s_values[4][2])]
     #float(s_values[5][2]),float(s_values[6][2])]
Tc = np.array(Tc)

inv_str_GeSiGe = np.reciprocal(str_GeSiGe)

inv_Tc = np.reciprocal(Tc)

#plot
fig = plt.figure(figsize=(12,8))

plt.plot(str_GeSiGe[:], Tc[:],'o-')


plt.xlabel(r'Length', fontdict={'fontsize':27})
plt.ylabel(r'k', fontdict={'fontsize':27})
plt.title(r'Projection of Convergence for Straight GeSiGe', fontdict={'fontsize':27})



fig = plt.figure(figsize=(12,8))

x1 = np.array(str_GeSiGe)
mn = min(x1)
mx = max(x1)

y1 = np.array(Tc)

z1 = np.polyfit(x1, y1, 1) #np.polyfit(x, y, degree of polynomial), degree =1,2,3 is linear, quadratic, cubic
z2 = np.polyfit(x1, y1, 2)
z3 = np.polyfit(x1, y1, 3)

xp = np.linspace(-0.001, mx+0.001, 6)

p1 = np.poly1d(z1)
p2 = np.poly1d(z2)
p3 = np.poly1d(z3)
#pz = np.poly1d(np.polyfit(x, y, 3)) #same as p3

plt.plot(x1, y1, 'o-', color ='black', label ='OG')
plt.plot(xp, p1(xp), 's', label = 'linear')
plt.plot(xp, p2(xp), 's', label = 'quadratic')
plt.plot(xp, p3(xp), 's', label ='cubic') 

plt.axvline(x=125.0, color=color_blind_safe[3])

plt.legend(loc="lower right")

plt.xlim([0.0,2000])
plt.ylim([0.0,20])

plt.xlabel(r'Length', fontdict={'fontsize':27})
plt.ylabel(r'k', fontdict={'fontsize':27})
plt.title(r'Projection of Convergence for Straight GeSiGe', fontdict={'fontsize':27})


fig = plt.figure(figsize=(12,8))

x_2 = np.array(str_GeSiGe)
mn = min(x_2)
mx = max(x_2)

y_2 = np.array(Tc)
xpp = np.linspace(-0.0005,mx, 6)

plt.axvline(x=125.0, color=color_blind_safe[3])


z3 = np.polyfit(x_2, y_2, 3)
p3 = np.poly1d(z3)
#pz = np.poly1d(np.polyfit(x, y, 3)) #same as p3

s = p3(1);
i = p3(2)    #The polyfit function for a linear (polynomial order 1) fit returns the slope as the first parameter
                    #and the intercept as the second parameter, so the output vector is [slope, intercept]. 
                    #It is like any other vector, so choose the one you want by indexing into it.
n = p3(0)
slope_intercept = np.polyfit(x_2,y_2,3)

plt.plot(x_2,y_2, 'o-', label ='OG')
plt.plot(xpp, p3(xp), 's', label ='cubic') 
plt.legend(loc="lower right")

plt.xlabel(r'Length', fontdict={'fontsize':27})
plt.ylabel(r'k', fontdict={'fontsize':27})
plt.title(r'Projection of Convergence for Straight GeSiGe cubic', fontdict={'fontsize':27})


#linear fit in linear region alone
fig = plt.figure(figsize=(12,8))

x2 = str_GeSiGe
x2 = np.array(x2)
y2 = Tc
y2 = np.array(y2)


z = np.polyfit(x2, y2, 1) 
xp = np.linspace(-0.001, mx+0.001, 3)

p1 = np.poly1d(z)

slope_intercept = np.polyfit(x2,y2,1)


plt.plot(x2, y2, 'o-', color ='black', label ='OG')
plt.plot(xp, p1(xp), 's', label = 'linear')

plt.axvline(x=0, color=color_blind_safe[3])

plt.legend(loc="lower right")

plt.xlabel(r'Length', fontdict={'fontsize':27})
plt.ylabel(r'k', fontdict={'fontsize':27})
plt.title(r'Projection of Convergence for linear Straight GeSiGe', fontdict={'fontsize':27})



SE200b_left = np.std([float(Thermals_str_GeSiGe[0][0]), float(Thermals_str_GeSiGe[0][2])], dtype=np.float64)
SE200_left = SE200b_left/math.sqrt(2)
SE200b_right = np.std([float(Thermals_str_GeSiGe[0][1]), float(Thermals_str_GeSiGe[0][3])], dtype=np.float64)
SE200_right = SE200b_right/math.sqrt(2)
standard_error200b_str_GeSiGe = math.sqrt(((SE200_left/2)**2) + ((SE200_right/2)**2))

SE400b_left = np.std([float(Thermals_str_GeSiGe[1][0]), float(Thermals_str_GeSiGe[1][2])], dtype=np.float64)
SE400_left = SE400b_left/math.sqrt(2)
SE400b_right = np.std([float(Thermals_str_GeSiGe[1][1]), float(Thermals_str_GeSiGe[1][3])], dtype=np.float64)
SE400_right = SE400b_right/math.sqrt(2)
standard_error400b_str_GeSiGe = math.sqrt(((SE400_left/2)**2) + ((SE400_right/2)**2))

SE600b_left = np.std([float(Thermals_str_GeSiGe[2][0]), float(Thermals_str_GeSiGe[2][2])], dtype=np.float64)
SE600_left = SE600b_left/math.sqrt(2)
SE600b_right = np.std([float(Thermals_str_GeSiGe[2][1]), float(Thermals_str_GeSiGe[2][3])], dtype=np.float64)
SE600_right = SE600b_right/math.sqrt(2)
standard_error600b_str_GeSiGe = math.sqrt(((SE600_left/2)**2) + ((SE600_right/2)**2))

SE800b_left = np.std([float(Thermals_str_GeSiGe[3][0]), float(Thermals_str_GeSiGe[3][2])], dtype=np.float64)
SE800_left = SE800b_left/math.sqrt(2)
SE800b_right = np.std([float(Thermals_str_GeSiGe[3][1]), float(Thermals_str_GeSiGe[3][3])], dtype=np.float64)
SE800_right = SE800b_right/math.sqrt(2)
standard_error800b_str_GeSiGe = math.sqrt(((SE800_left/2)**2) + ((SE800_right/2)**2))

SE1000b_left = np.std([float(Thermals_str_GeSiGe[4][0]), float(Thermals_str_GeSiGe[4][2])], dtype=np.float64)
SE1000_left = SE1000b_left/math.sqrt(2)
SE1000b_right = np.std([float(Thermals_str_GeSiGe[4][1]), float(Thermals_str_GeSiGe[4][3])], dtype=np.float64)
SE1000_right = SE1000b_right/math.sqrt(2)
standard_error1000b_str_GeSiGe = math.sqrt(((SE1000_left/2)**2) + ((SE1000_right/2)**2))

yerr200b_str_GeSiGe = standard_error200b_str_GeSiGe
yerr400b_str_GeSiGe = standard_error400b_str_GeSiGe
yerr600b_str_GeSiGe = standard_error600b_str_GeSiGe
yerr800b_str_GeSiGe = standard_error800b_str_GeSiGe
yerr1000b_str_GeSiGe = standard_error1000b_str_GeSiGe


In [ ]:
filename = ['./profile_siE200r50', 'profile_siE400r50', 'profile_siE600r50',
           'profile_siE800r50', 'profile_siE1000r50']



MP_Profile = []; MP_Ave = []; MP_LastN = []; 

for simNo in range(0,len(filename)):
    subprocess.call(r"perl -p -e 's/^[\ \t]+|[\ \t]+$//g' %s > %s_new" % (filename[simNo], filename[simNo]), shell=True) # remove trailling white spaces
    subprocess.call("mv %s_new %s" % (filename[simNo], filename[simNo]), shell=True)
    subprocess.call("tr -s ' ' < %s > %s_tmp " % (filename[simNo], filename[simNo]), shell=True) #squeeze spaces
    subprocess.call("mv %s_tmp %s" % (filename[simNo], filename[simNo]), shell=True)

    profile = []; N = 800;
    with open(filename[simNo], 'r') as f:
        while True:
            lines = list(itertools.islice(f, 1, N+1))
            for line in lines:
                profile.append(line.split(' '))
            if not lines: 
                break
                
    lastN = profile[-N:] 
    lastAve = profile[-3200:]

    MP_Profile.append(profile)
    MP_LastN.append(lastN)
    MP_Ave.append(lastAve)
    
MP_Profile = np.array(MP_Profile, dtype=float) # for plotting 3D graphs

MP_LastN = np.array(MP_LastN, dtype=float) # for calculating the thermal conductivity



MP_Ave = np.array(MP_Ave, dtype=float) # for averaging the data to calculate the thermal conductivity

Thermo_Data = np.array([np.loadtxt('./Thermo_siE200r50'), np.loadtxt('./Thermo_siE400r50'), np.loadtxt('./Thermo_siE600r50'),
                       np.loadtxt('./Thermo_siE800r50'), np.loadtxt('./Thermo_siE1000r50')], dtype = object)

labels = ['siE200', 'siE400', 'siE600', 'siE800', 'siE1000','siE1500','siE2000'] 


for simNo in range(0, len(MP_Profile)):
    Coordinates = MP_Profile[simNo,:,1]*(Thermo_Data[simNo,-1,9])
    Temperature = MP_Profile[simNo,:,3]

fig = plt.figure(figsize=(12,8))
for k in range(0, len(MP_Profile)):
    plt.plot(Coordinates[:], Temperature[:], color=colors[k], label=labels[k])
    
plt.xlabel(r'Position in Supercell ($\AA$)')
plt.ylabel('Temperature (K)')
plt.title('Muller-Plathe Temperature Gradient', fontsize = 24)

# Load the data that will be averaged to calculate the slopes 

Temperature = []; Coordinate = []; dX_Thermo = [];

for simNo in range(0, len(filename)):
    
    dX_ = Thermo_Data[simNo,-1,9]
    dX_Thermo.append(dX_)
    
    for i in range(0,800):
        Temperature_ = np.mean([MP_Ave[simNo,i,3],MP_Ave[simNo,i+800,3],MP_Ave[simNo,i+1600,3],MP_Ave[simNo,i+2400,3]])   
        Temperature.append(Temperature_)
        Coordinate_ = np.mean([MP_Ave[simNo,i,1],MP_Ave[simNo,i+800,1],MP_Ave[simNo,i+1600,1],MP_Ave[simNo,i+2400,1]])*dX_
        Coordinate.append(Coordinate_)
    
Temperature = np.array([Temperature[0:800],Temperature[800:1600],Temperature[1600:2400],
                       Temperature[2400:3200],Temperature[3200:4000]], dtype=float)
                     
Coordinate = np.array([Coordinate[0:800],Coordinate[800:1600], Coordinate[1600:2400],
                      Coordinate[2400:3200],Coordinate[3200:4000],], dtype=float)
                      



excluded_points1 = 230 # excluded points from the beginning
excluded_points2 = 40 # excluded points from the end
excluded_points3 = 20
excluded_points4 = 230 # from the end of the mp.data file

slope2_mean = []; slope2 = []; slope1_mean = []; slope1 = []; average_temp_Si50 = [];

# Step Temp E_pair PotEng KinEng TotEng Temp Press Volume Lx Ly Lz f_3 v_tdiff
for simNo in range(0,len(filename)):
    
    dTemp = Temperature[simNo,:]
    X = dX_
    dX = Coordinate[simNo,:] #*X # normalized distance
    
    T_fit_left = np.polyfit(dX[excluded_points1:400-excluded_points2], dTemp[excluded_points1:400-excluded_points2], 1)
    T_fit_right = -np.polyfit(dX[400+excluded_points2:-excluded_points1], dTemp[400+excluded_points2:-excluded_points1], 1)   
    slope2.append([T_fit_left[0], T_fit_right[0]])
    mean_slope = np.mean([abs(T_fit_left[0]), abs(T_fit_right[0])], axis = 0)
    slope2_mean.append(mean_slope)
    B_fit_left2 = np.polyfit(dX[excluded_points3:400-excluded_points4], dTemp[excluded_points3:400-excluded_points4], 1)
    B_fit_right2 = -np.polyfit(dX[400+excluded_points4:-excluded_points3], dTemp[400+excluded_points4:-excluded_points3], 1)    
    slope1.append([B_fit_left2[0], B_fit_right2[0]])
    mean_slope = np.mean([abs(B_fit_left2[0]), abs(B_fit_right2[0])], axis = 0)
    slope1_mean.append(mean_slope)
    average_temp = np.mean(dTemp)
    average_temp_Si50.append(average_temp)

Slope1 = np.array(slope1)
Slope_mean1 = np.array(slope1_mean)

Slope2 = np.array(slope2)
Slope_mean2 = np.array(slope2_mean)

Slope_mean = np.array([np.mean([Slope_mean1[0], Slope_mean2[0]], axis=0)]) #individual
#Slope_mean = np.array([np.mean([Slope_mean1[0], Slope_mean2[0]], axis=0), np.mean([Slope_mean1[1], Slope_mean2[1]], axis =0), np.mean([Slope_mean1[2], Slope_mean2[2]], axis=0)])

Slope_ave = np.array([Slope_mean[0]*dX_Thermo[0]]) #individual
#Slope_ave = np.array([Slope_mean[0]*dX_Thermo[0], Slope_mean[1]*dX_Thermo[1], Slope_mean[2]*dX_Thermo[2]])

# Calculate the heat flux and the thermal conductivity
eV2J  = 1.6e-19;
Ang2m =   1e-10;
ps2s  =   1e-12;
conversion = (eV2J/(Ang2m*ps2s))

s_values = []
ThermalsSi50 = []
dt = 0.0005 # picoseconds

for simNo in range(0, len(Thermo_Data)):
    
    Steps = Thermo_Data[simNo,-1,0]
    Time = Steps*dt # picoseconds
    
    dx = Thermo_Data[simNo,-1,9]
    dy = Thermo_Data[simNo,-1,10]
    dz = Thermo_Data[simNo,-1,11]
    Area = dy*dz
    
    #DeltaE = Thermo_Data[simNo,-1,-2] # E_pair
    #TotalE = Thermo_Data[simNo,-1,-5] # Total energy
    KinE = Thermo_Data[simNo,-1,12] # in log_lammps, f_3 
    
    Thermal_left2 = ((KinE)/(2*Time*Area*Slope2[simNo,0]))*conversion #top left
    Thermal_right2 = ((KinE)/(2*Time*Area*Slope2[simNo,1]))*conversion #top right
    
    Thermal_left1 = ((KinE)/(2*Time*Area*Slope1[simNo,0]))*conversion #bottom left
    Thermal_right1 = ((KinE)/(2*Time*Area*Slope1[simNo,1]))*conversion #bottom right
    
    thermal2 = np.concatenate((Thermal_left1, Thermal_left2), axis = None)
    thermal1 = np.concatenate((Thermal_right1, Thermal_right2), axis = None)
    
    thermal2_ave = ((KinE)/(2*Time*Area*Slope_mean2[simNo]))*conversion #top
    thermal1_ave = ((KinE)/(2*Time*Area*Slope_mean1[simNo]))*conversion #bottom
    
    #thermal_ave = ((KinE)/(2*Time*Area*Slope_ave[simNo]))*conversion
    thermal_ave = (thermal2_ave+thermal1_ave)/2
   
    getcontext().prec = 10
    std_top = Decimal(np.std([Thermal_left2, Thermal_right2], dtype=np.float64))
    SE_left = std_top/Decimal(math.sqrt(2))
   
    std_bottom = Decimal(np.std([Thermal_left1, Thermal_right1], dtype=np.float64))
    SE_right = std_bottom/Decimal(math.sqrt(2))
    
    SE = math.sqrt(((SE_left/2)**2) + ((SE_right/2)**2))
    
    Therm_val = [Thermal_left1,Thermal_left2,Thermal_right2,Thermal_right1]
    Ttherm = ['%.6f' % elem for elem in Therm_val]
    ThermalsSi50.append(Ttherm)
    
    
    TC_values = [thermal2_ave, thermal1_ave, thermal_ave, SE]
    val = ['%.6f' % elem for elem in TC_values]
    s_values.append(val)
 

therm_averageSi50 = []
for item in s_values:
    therm_averageSi50.append(item[2])

therm_averageSi50 = []
for item in s_values:
    therm_averageSi50.append(np.array(float(item[2])))

#projection stuff
#SIGESI r50
sizeSi50 = [200.0,400.0,600.0,800.0,1000.0]#,1500.0,2000.0]
sizeSi50 = np.array(sizeSi50)
Tc = [float(s_values[0][2]),float(s_values[1][2]),float(s_values[2][2]),
      float(s_values[3][2]),float(s_values[4][2])]
      #,float(s_values[5][2]),float(s_values[6][2])]
Tc = np.array(Tc)

inv_sizeSi50 = np.reciprocal(sizeSi50)
inv_Tc = np.reciprocal(Tc)

#plot
fig = plt.figure(figsize=(12,8))

plt.plot(sizeSi50[:], Tc[:],'o-')


plt.xlabel(r'Length', fontdict={'fontsize':27})
plt.ylabel(r'k', fontdict={'fontsize':27})
plt.title(r'Projection of Convergence for SiGeSi, r50', fontdict={'fontsize':27})


fig = plt.figure(figsize=(12,8))

x1 = np.array(sizeSi50)
mn = min(x1)
mx = max(x1)

y1 = np.array(Tc)

z1 = np.polyfit(x1, y1, 1) #np.polyfit(x, y, degree of polynomial), degree =1,2,3 is linear, quadratic, cubic
z2 = np.polyfit(x1, y1, 2)
z3 = np.polyfit(x1, y1, 3)

xp = np.linspace(-0.001, mx+0.001, 6)

p1 = np.poly1d(z1)
p2 = np.poly1d(z2)
p3 = np.poly1d(z3)
#pz = np.poly1d(np.polyfit(x, y, 3)) #same as p3

plt.plot(x1, y1, 'o-', color ='black', label ='OG')
plt.plot(xp, p1(xp), 's', label = 'linear')
plt.plot(xp, p2(xp), 's', label = 'quadratic')
plt.plot(xp, p3(xp), 's', label ='cubic') 

plt.axvline(x=50.0, color=color_blind_safe[3])

plt.legend(loc="lower right")

plt.xlim([0.0,2000])
plt.ylim([0.0,20])

plt.xlabel(r'Length', fontdict={'fontsize':27})
plt.ylabel(r'k', fontdict={'fontsize':27})
plt.title(r'Projection of Convergence for SiGeSi, r50', fontdict={'fontsize':27})


fig = plt.figure(figsize=(12,8))

x_2 = np.array(sizeSi50)
mn = min(x_2)
mx = max(x_2)

y_2 = np.array(Tc)

xpp = np.linspace(-0.0005,mx, 6)

plt.axvline(x=50.0, color=color_blind_safe[3])


z3 = np.polyfit(x_2, y_2, 3)
p3 = np.poly1d(z3)
s = p3(1);
i = p3(2)    #The polyfit function for a linear (polynomial order 1) fit returns the slope as the first parameter
                    #and the intercept as the second parameter, so the output vector is [slope, intercept]. 
                    #It is like any other vector, so choose the one you want by indexing into it.
n = p3(0)
slope_intercept = np.polyfit(x_2,y_2,3)

plt.plot(x_2,y_2, 'o-', label ='OG')
plt.plot(xpp, p3(xp), 's', label ='cubic') 
plt.legend(loc="lower right")

plt.xlabel(r'Length', fontdict={'fontsize':27})
plt.ylabel(r'k', fontdict={'fontsize':27})
plt.title(r'Projection of Convergence for SiGeSi, r50 cubic', fontdict={'fontsize':27})


#linear fit in linear region alone
fig = plt.figure(figsize=(12,8))

x2 = sizeSi50
x2 = np.array(x2)
y2 = Tc
y2 = np.array(y2)


z = np.polyfit(x2, y2, 1) 
xp = np.linspace(-0.001, mx+0.001, 3)

p1 = np.poly1d(z)

slope_intercept = np.polyfit(x2,y2,1)

plt.plot(x2, y2, 'o-', color ='black', label ='OG')
plt.plot(xp, p1(xp), 's', label = 'linear')

plt.axvline(x=0, color=color_blind_safe[3])

plt.legend(loc="lower right")

plt.xlabel(r'Length', fontdict={'fontsize':27})
plt.ylabel(r'k', fontdict={'fontsize':27})
plt.title(r'Projection of Convergence for linear SiGeSi, r50', fontdict={'fontsize':27})

#SiGeSi values

SE200b_left = np.std([float(ThermalsSi50[0][1]), float(ThermalsSi50[0][2])], dtype=np.float64)
SE200_left = SE200b_left/math.sqrt(2)
SE200b_right = np.std([float(ThermalsSi50[0][0]), float(ThermalsSi50[0][3])], dtype=np.float64)
SE200_right = SE200b_right/math.sqrt(2)
standard_error200b50 = math.sqrt(((SE200_left/2)**2) + ((SE200_right/2)**2))

SE400b_left = np.std([float(ThermalsSi50[1][1]), float(ThermalsSi50[1][2])], dtype=np.float64)
SE400_left = SE400b_left/math.sqrt(2)
SE400b_right = np.std([float(ThermalsSi50[1][0]), float(ThermalsSi50[1][3])], dtype=np.float64)
SE400_right = SE400b_right/math.sqrt(2)
standard_error400b50 = math.sqrt(((SE400_left/2)**2) + ((SE400_right/2)**2))

SE600b_left = np.std([float(ThermalsSi50[2][1]), float(ThermalsSi50[2][2])], dtype=np.float64)
SE600_left = SE600b_left/math.sqrt(2)
SE600b_right = np.std([float(ThermalsSi50[2][0]), float(ThermalsSi50[2][3])], dtype=np.float64)
SE600_right = SE600b_right/math.sqrt(2)
standard_error600b50 = math.sqrt(((SE600_left/2)**2) + ((SE600_right/2)**2))


SE800b_left = np.std([float(ThermalsSi50[3][1]), float(ThermalsSi50[3][2])], dtype=np.float64)
SE800_left = SE800b_left/math.sqrt(2)
SE800b_right = np.std([float(ThermalsSi50[3][0]), float(ThermalsSi50[3][3])], dtype=np.float64)
SE800_right = SE800b_right/math.sqrt(2)
standard_error800b50 = math.sqrt(((SE800_left/2)**2) + ((SE800_right/2)**2))

SE1000b_left = np.std([float(ThermalsSi50[4][1]), float(ThermalsSi50[4][2])], dtype=np.float64)
SE1000_left = SE1000b_left/math.sqrt(2)
SE1000b_right = np.std([float(ThermalsSi50[4][0]), float(ThermalsSi50[4][3])], dtype=np.float64)
SE1000_right = SE1000b_right/math.sqrt(2)
standard_error1000b50 = math.sqrt(((SE1000_left/2)**2) + ((SE1000_right/2)**2))

yerrSi200b50 = standard_error200b50
yerrSi400b50 = standard_error400b50
yerrSi600b50 = standard_error600b50
yerrSi800b50 = standard_error800b50
yerrSi1000b50 = standard_error1000b50



In [ ]:
#50

In [ ]:
filename = ['./profile_geE200r50','profile_geE400r50','profile_geE600r50',
           'profile_geE800r50','profile_geE1000r50']



MP_Profile = []; MP_Ave = []; MP_LastN = []; 

for simNo in range(0,len(filename)):
    subprocess.call(r"perl -p -e 's/^[\ \t]+|[\ \t]+$//g' %s > %s_new" % (filename[simNo], filename[simNo]), shell=True) # remove trailling white spaces
    subprocess.call("mv %s_new %s" % (filename[simNo], filename[simNo]), shell=True)
    subprocess.call("tr -s ' ' < %s > %s_tmp " % (filename[simNo], filename[simNo]), shell=True) #squeeze spaces
    subprocess.call("mv %s_tmp %s" % (filename[simNo], filename[simNo]), shell=True)

    profile = []; N = 800;
    with open(filename[simNo], 'r') as f:
        while True:
            lines = list(itertools.islice(f, 1, N+1))
            for line in lines:
                profile.append(line.split(' '))
            if not lines: 
                break
                
    lastN = profile[-N:] 
    lastAve = profile[-3200:]

    MP_Profile.append(profile)
    MP_LastN.append(lastN)
    MP_Ave.append(lastAve)
    
MP_Profile = np.array(MP_Profile, dtype=float) # for plotting 3D graphs

MP_LastN = np.array(MP_LastN, dtype=float) # for calculating the thermal conductivity



MP_Ave = np.array(MP_Ave, dtype=float) # for averaging the data to calculate the thermal conductivity

Thermo_Data = np.array([np.loadtxt('./Thermo_geE200r50'),np.loadtxt('./Thermo_geE400r50'),np.loadtxt('./Thermo_geE600r50'),
                       np.loadtxt('./Thermo_geE800r50'),np.loadtxt('./Thermo_geE1000r50')], dtype = object)#
                        

labels = ['geE200', 'geE400', 'geE600', 'geE800','geE1000','geE1500', 'ge2000'] 


for simNo in range(0, len(MP_Profile)):
    Coordinates = MP_Profile[simNo,:,1]*(Thermo_Data[simNo,-1,9])
    Temperature = MP_Profile[simNo,:,3]
 
fig = plt.figure(figsize=(12,8))
for k in range(0, len(MP_Profile)):
    plt.plot(Coordinates[:], Temperature[:], color=colors[k], label=labels[k])
    
plt.xlabel(r'Position in Supercell ($\AA$)')
plt.ylabel('Temperature (K)')
plt.title('Muller-Plathe Temperature Gradient', fontsize = 24)

# Load the data that will be averaged to calculate the slopes 
Temperature = []; Coordinate = []; dX_Thermo = [];

for simNo in range(0, len(filename)):
    
    dX_ = Thermo_Data[simNo,-1,9]
    dX_Thermo.append(dX_)
    
    for i in range(0,800):
        Temperature_ = np.mean([MP_Ave[simNo,i,3],MP_Ave[simNo,i+800,3],MP_Ave[simNo,i+1600,3],MP_Ave[simNo,i+2400,3]])   
        Temperature.append(Temperature_)
        Coordinate_ = np.mean([MP_Ave[simNo,i,1],MP_Ave[simNo,i+800,1],MP_Ave[simNo,i+1600,1],MP_Ave[simNo,i+2400,1]])*dX_
        Coordinate.append(Coordinate_)
    
Temperature = np.array([Temperature[0:800],Temperature[800:1600],Temperature[1600:2400],
                        Temperature[2400:3200],Temperature[3200:4000]], dtype=float)
                        
Coordinate = np.array([Coordinate[0:800],Coordinate[800:1600],Coordinate[1600:2400],
                      Coordinate[2400:3200],Coordinate[3200:4000]], dtype=float)
                     




excluded_points1 = 230 # excluded points from the beginning
excluded_points2 = 40 # excluded points from the end
excluded_points3 = 20
excluded_points4 = 230 # from the end of the mp.data file

slope2_mean = []; slope2 = []; slope1_mean = []; slope1 = []; average_temp_Ge50 = [];

for simNo in range(0,len(filename)):
    
    dTemp = Temperature[simNo,:]
    X = dX_
    dX = Coordinate[simNo,:] #*X # normalized distance
        
    T_fit_left = np.polyfit(dX[excluded_points1:400-excluded_points2], dTemp[excluded_points1:400-excluded_points2], 1)
    T_fit_right = -np.polyfit(dX[400+excluded_points2:-excluded_points1], dTemp[400+excluded_points2:-excluded_points1], 1)    
    slope2.append([T_fit_left[0], T_fit_right[0]])
    mean_slope = np.mean([abs(T_fit_left[0]), abs(T_fit_right[0])], axis = 0)
    slope2_mean.append(mean_slope)
    B_fit_left2 = np.polyfit(dX[excluded_points3:400-excluded_points4], dTemp[excluded_points3:400-excluded_points4], 1)
    B_fit_right2 = -np.polyfit(dX[400+excluded_points4:-excluded_points3], dTemp[400+excluded_points4:-excluded_points3], 1)
    slope1.append([B_fit_left2[0], B_fit_right2[0]])
    mean_slope = np.mean([abs(B_fit_left2[0]), abs(B_fit_right2[0])], axis = 0)
    slope1_mean.append(mean_slope)
    average_temp = np.mean(dTemp)
    average_temp_Ge50.append(average_temp)

Slope1 = np.array(slope1)
Slope_mean1 = np.array(slope1_mean)

Slope2 = np.array(slope2)
Slope_mean2 = np.array(slope2_mean)

Slope_mean = np.array([np.mean([Slope_mean1[0], Slope_mean2[0]], axis=0)]) #individual
#Slope_mean = np.array([np.mean([Slope_mean1[0], Slope_mean2[0]], axis=0), np.mean([Slope_mean1[1], Slope_mean2[1]], axis =0), np.mean([Slope_mean1[2], Slope_mean2[2]], axis=0)])

Slope_ave = np.array([Slope_mean[0]*dX_Thermo[0]]) #individual
#Slope_ave = np.array([Slope_mean[0]*dX_Thermo[0], Slope_mean[1]*dX_Thermo[1], Slope_mean[2]*dX_Thermo[2]])

# Calculate the heat flux and the thermal conductivity
eV2J  = 1.6e-19;
Ang2m =   1e-10;
ps2s  =   1e-12;
conversion = (eV2J/(Ang2m*ps2s))

s_values = []
ThermalsGe50 = []
dt = 0.0005 # picoseconds

for simNo in range(0, len(Thermo_Data)):
    
    Steps = Thermo_Data[simNo,-1,0]
    Time = Steps*dt # picoseconds
    
    dx = Thermo_Data[simNo,-1,9]
    dy = Thermo_Data[simNo,-1,10]
    dz = Thermo_Data[simNo,-1,11]
    Area = dy*dz
    
    #DeltaE = Thermo_Data[simNo,-1,-2] # E_pair
    #TotalE = Thermo_Data[simNo,-1,-5] # Total energy
    KinE = Thermo_Data[simNo,-1,12] # in log_lammps, f_3 
    
    Thermal_left2 = ((KinE)/(2*Time*Area*Slope2[simNo,0]))*conversion #top left
    Thermal_right2 = ((KinE)/(2*Time*Area*Slope2[simNo,1]))*conversion #top right
    
    Thermal_left1 = ((KinE)/(2*Time*Area*Slope1[simNo,0]))*conversion #bottom left
    Thermal_right1 = ((KinE)/(2*Time*Area*Slope1[simNo,1]))*conversion #bottom right
    
    thermal2 = np.concatenate((Thermal_left1, Thermal_left2), axis = None)
    thermal1 = np.concatenate((Thermal_right1, Thermal_right2), axis = None)
    
    thermal2_ave = ((KinE)/(2*Time*Area*Slope_mean2[simNo]))*conversion #top
    thermal1_ave = ((KinE)/(2*Time*Area*Slope_mean1[simNo]))*conversion #bottom
    
    #thermal_ave = ((KinE)/(2*Time*Area*Slope_ave[simNo]))*conversion
    thermal_ave = (thermal2_ave+thermal1_ave)/2
    
    getcontext().prec = 10
    std_top = Decimal(np.std([Thermal_left2, Thermal_right2], dtype=np.float64))
    SE_left = std_top/Decimal(math.sqrt(2))
   
    std_bottom = Decimal(np.std([Thermal_left1, Thermal_right1], dtype=np.float64))
    SE_right = std_bottom/Decimal(math.sqrt(2))
    
    SE = math.sqrt(((SE_left/2)**2) + ((SE_right/2)**2))

    Therm_val = [Thermal_left1,Thermal_left2,Thermal_right2,Thermal_right1]
    Ttherm = ['%.6f' % elem for elem in Therm_val]

    ThermalsGe50.append(Ttherm)
    
    
    TC_values = [thermal2_ave, thermal1_ave, thermal_ave, SE]
    val = ['%.6f' % elem for elem in TC_values]
    s_values.append(val)
    

therm_averageGe50 = []
for item in s_values:
    therm_averageGe50.append(item[2])

therm_averageGe50 = []
for item in s_values:
    therm_averageGe50.append(np.array(float(item[2])))

#projection stuff

sizeGe50 = [200.0,400.0,600.0,800.0,1000.0]
sizeGe50 = np.array(sizeGe50)
Tc = [float(s_values[0][2]),float(s_values[1][2]),float(s_values[2][2]),
     float(s_values[3][2]),float(s_values[4][2])]#
Tc = np.array(Tc)

inv_sizeGe50 = np.reciprocal(sizeGe50)

inv_Tc = np.reciprocal(Tc)

#plot
fig = plt.figure(figsize=(12,8))

plt.plot(sizeGe50[:], Tc[:],'o-')


plt.xlabel(r'Length', fontdict={'fontsize':27})
plt.ylabel(r'k', fontdict={'fontsize':27})
plt.title(r'Projection of Convergence for SiGeSi, r50', fontdict={'fontsize':27})


fig = plt.figure(figsize=(12,8))

x1 = np.array(sizeGe50)
mn = min(x1)
mx = max(x1)

y1 = np.array(Tc)

z1 = np.polyfit(x1, y1, 1) #np.polyfit(x, y, degree of polynomial), degree =1,2,3 is linear, quadratic, cubic
z2 = np.polyfit(x1, y1, 2)
z3 = np.polyfit(x1, y1, 3)

xp = np.linspace(-0.001, mx+0.001, 6)

p1 = np.poly1d(z1)
p2 = np.poly1d(z2)
p3 = np.poly1d(z3)
#pz = np.poly1d(np.polyfit(x, y, 3)) #same as p3

plt.plot(x1, y1, 'o-', color ='black', label ='OG')
plt.plot(xp, p1(xp), 's', label = 'linear')
plt.plot(xp, p2(xp), 's', label = 'quadratic')
plt.plot(xp, p3(xp), 's', label ='cubic') 

plt.axvline(x=50.0, color=color_blind_safe[3])

plt.legend(loc="lower right")

plt.xlim([0.0,2000])
plt.ylim([0.0,20])

plt.xlabel(r'Length', fontdict={'fontsize':27})
plt.ylabel(r'k', fontdict={'fontsize':27})
plt.title(r'Projection of Convergence for SiGeSi, r50', fontdict={'fontsize':27})


fig = plt.figure(figsize=(12,8))

x_2 = np.array(sizeGe50)
mn = min(x_2)
mx = max(x_2)
y_2 = np.array(Tc)
xpp = np.linspace(-0.0005,mx, 6)

plt.axvline(x=50.0, color=color_blind_safe[3])


z3 = np.polyfit(x_2, y_2, 3)
p3 = np.poly1d(z3)
#pz = np.poly1d(np.polyfit(x, y, 3)) #same as p3

s = p3(1);
i = p3(2)    #The polyfit function for a linear (polynomial order 1) fit returns the slope as the first parameter
                    #and the intercept as the second parameter, so the output vector is [slope, intercept]. 
                    #It is like any other vector, so choose the one you want by indexing into it.
n = p3(0)
slope_intercept = np.polyfit(x_2,y_2,3)

plt.plot(x_2,y_2, 'o-', label ='OG')
plt.plot(xpp, p3(xp), 's', label ='cubic') 
plt.legend(loc="lower right")

plt.xlabel(r'Length', fontdict={'fontsize':27})
plt.ylabel(r'k', fontdict={'fontsize':27})
plt.title(r'Projection of Convergence for SiGeSi, r50 cubic', fontdict={'fontsize':27})

#linear fit in linear region alone
fig = plt.figure(figsize=(12,8))

x2 = sizeGe50
x2 = np.array(x2)
y2 = Tc
y2 = np.array(y2)


z = np.polyfit(x2, y2, 1) 
xp = np.linspace(-0.001, mx+0.001, 3)

p1 = np.poly1d(z)

slope_intercept = np.polyfit(x2,y2,1)


plt.plot(x2, y2, 'o-', color ='black', label ='OG')
plt.plot(xp, p1(xp), 's', label = 'linear')

plt.axvline(x=0, color=color_blind_safe[3])

plt.legend(loc="lower right")

plt.xlabel(r'Length', fontdict={'fontsize':27})
plt.ylabel(r'k', fontdict={'fontsize':27})
plt.title(r'Projection of Convergence for linear SiGeSi, r50', fontdict={'fontsize':27})


SE200b_left = np.std([float(ThermalsGe50[0][1]), float(ThermalsGe50[0][2])], dtype=np.float64)
SE200_left = SE200b_left/math.sqrt(2)
SE200b_right = np.std([float(ThermalsGe50[0][0]), float(ThermalsGe50[0][3])], dtype=np.float64)
SE200_right = SE200b_right/math.sqrt(2)
standard_error200b50 = math.sqrt(((SE200_left/2)**2) + ((SE200_right/2)**2))

SE400b_left = np.std([float(ThermalsGe50[1][1]), float(ThermalsGe50[1][2])], dtype=np.float64)
SE400_left = SE400b_left/math.sqrt(2)
SE400b_right = np.std([float(ThermalsGe50[1][0]), float(ThermalsGe50[1][3])], dtype=np.float64)
SE400_right = SE400b_right/math.sqrt(2)
standard_error400b50 = math.sqrt(((SE400_left/2)**2) + ((SE400_right/2)**2))

SE600b_left = np.std([float(ThermalsGe50[2][1]), float(ThermalsGe50[2][2])], dtype=np.float64)
SE600_left = SE600b_left/math.sqrt(2)
SE600b_right = np.std([float(ThermalsGe50[2][0]), float(ThermalsGe50[2][3])], dtype=np.float64)
SE600_right = SE600b_right/math.sqrt(2)
standard_error600b50 = math.sqrt(((SE600_left/2)**2) + ((SE600_right/2)**2))


SE800b_left = np.std([float(ThermalsGe50[3][1]), float(ThermalsGe50[3][2])], dtype=np.float64)
SE800_left = SE800b_left/math.sqrt(2)
SE800b_right = np.std([float(ThermalsGe50[3][0]), float(ThermalsGe50[3][3])], dtype=np.float64)
SE800_right = SE800b_right/math.sqrt(2)
standard_error800b50 = math.sqrt(((SE800_left/2)**2) + ((SE800_right/2)**2))

SE1000b_left = np.std([float(ThermalsGe50[4][1]), float(ThermalsGe50[4][2])], dtype=np.float64)
SE1000_left = SE1000b_left/math.sqrt(2)
SE1000b_right = np.std([float(ThermalsGe50[4][0]), float(ThermalsGe50[4][3])], dtype=np.float64)
SE1000_right = SE1000b_right/math.sqrt(2)
standard_error1000b50 = math.sqrt(((SE1000_left/2)**2) + ((SE1000_right/2)**2))

yerrGe200b50 = standard_error200b50
yerrGe400b50 = standard_error400b50
yerrGe600b50 = standard_error600b50
yerrGe800b50 = standard_error800b50
yerrGe1000b50 = standard_error1000b50


In [ ]:
b = ['110.0','219.9','329.9','439.9','550.0']
b = np.array(b)
size_prstG = b

size_prst = b

str_SiGeSi = b

str_GeSiGe = b

sizeSi50 = b

sizeSi125 = b

sizeSi250 = b

sizeSi271 = b

sizeGe50 = b

sizeGe125 = b

sizeGe250 = b

sizeGe271 = b


In [ ]:
fig = plt.figure(figsize=(12,8))

#PristineSi
plt.plot(size_prst[:], np.array(therm_average_prst[:]),'o-', label = 'Pristine Si',markerfacecolor='none', markeredgecolor=color_blind_safe[5],color=color_blind_safe[5])

plt.errorbar(size_prst[0], therm_average_prst[0], yerr200b_prst, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[5],color=color_blind_safe[5])
plt.errorbar(size_prst[1], therm_average_prst[1], yerr400b_prst, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[5],color=color_blind_safe[5]) 
plt.errorbar(size_prst[2], therm_average_prst[2], yerr600b_prst, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[5],color=color_blind_safe[5]) 
plt.errorbar(size_prst[3], therm_average_prst[3], yerr800b_prst, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[5],color=color_blind_safe[5]) 
plt.errorbar(size_prst[4], therm_average_prst[4], yerr1000b_prst, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[5],color=color_blind_safe[5]) 

plt.legend(bbox_to_anchor=(1.1, 1.0), loc='upper left',labelspacing=1.0,fontsize=20) #,prop={'size': 20}
plt.xlabel('Length (nm)', fontdict={'fontsize':27})
plt.ylabel(r'Thermal conductivity (W/m·K)', fontdict={'fontsize':27})
#plt.title(r'Projection of Direct Convergence for all Systems', fontdict={'fontsize':27})

plt.show

In [ ]:
fig = plt.figure(figsize=(12,8))

#Straight SiGeSi
plt.plot(str_SiGeSi[:], np.array(therm_average_str_SiGeSi[:]),'o-', label = 'Ge|hot|',markerfacecolor='none', markeredgecolor=color_blind_safe[0],color=color_blind_safe[0])

plt.errorbar(str_SiGeSi[0], therm_average_str_SiGeSi[0], yerr200b_str_SiGeSi, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[0],color=color_blind_safe[0])
plt.errorbar(str_SiGeSi[1], therm_average_str_SiGeSi[1], yerr400b_str_SiGeSi, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[0],color=color_blind_safe[0]) 
plt.errorbar(str_SiGeSi[2], therm_average_str_SiGeSi[2], yerr600b_str_SiGeSi, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[0],color=color_blind_safe[0]) 
plt.errorbar(str_SiGeSi[3], therm_average_str_SiGeSi[3], yerr800b_str_SiGeSi, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[0],color=color_blind_safe[0]) 
plt.errorbar(str_SiGeSi[4], therm_average_str_SiGeSi[4], yerr1000b_str_SiGeSi, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[0],color=color_blind_safe[0]) 

plt.legend(bbox_to_anchor=(1.1, 1.0), loc='upper left',labelspacing=1.0,fontsize=20) #,prop={'size': 20}
plt.xlabel('Length (nm)', fontdict={'fontsize':27})
plt.ylabel(r'Thermal conductivity (W/m·K)', fontdict={'fontsize':27})
#plt.title(r'Projection of Direct Convergence for all Systems', fontdict={'fontsize':27})

plt.show

In [ ]:
fig = plt.figure(figsize=(12,8))

#Straight SiGeSi
plt.plot(str_SiGeSi[:], np.array(therm_average_str_SiGeSi[:]),'o-', label = 'Ge|hot|',markerfacecolor='none', markeredgecolor=color_blind_safe[0],color=color_blind_safe[0])

plt.errorbar(str_SiGeSi[0], therm_average_str_SiGeSi[0], yerr200b_str_SiGeSi, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[0],color=color_blind_safe[0])
plt.errorbar(str_SiGeSi[1], therm_average_str_SiGeSi[1], yerr400b_str_SiGeSi, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[0],color=color_blind_safe[0]) 
plt.errorbar(str_SiGeSi[2], therm_average_str_SiGeSi[2], yerr600b_str_SiGeSi, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[0],color=color_blind_safe[0]) 
plt.errorbar(str_SiGeSi[3], therm_average_str_SiGeSi[3], yerr800b_str_SiGeSi, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[0],color=color_blind_safe[0]) 
plt.errorbar(str_SiGeSi[4], therm_average_str_SiGeSi[4], yerr1000b_str_SiGeSi, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[0],color=color_blind_safe[0]) 

#Si
#Si50
plt.plot(sizeSi50[:], np.array(therm_averageSi50[:]),'o-', label='Ge(hot)-r50',markerfacecolor='none', markeredgecolor=color_blind_safe[1],color=color_blind_safe[1])

plt.errorbar(sizeSi50[0], therm_averageSi50[0], yerrSi200b50, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[1],color=color_blind_safe[1])
plt.errorbar(sizeSi50[1], therm_averageSi50[1], yerrSi400b50, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[1],color=color_blind_safe[1]) 
plt.errorbar(sizeSi50[2], therm_averageSi50[2], yerrSi600b50, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[1],color=color_blind_safe[1]) 
plt.errorbar(sizeSi50[3], therm_averageSi50[3], yerrSi800b50, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[1],color=color_blind_safe[1]) 
plt.errorbar(sizeSi50[4], therm_averageSi50[4], yerrSi1000b50, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[1],color=color_blind_safe[1]) 

#Si125
plt.plot(sizeSi125[:], np.array(therm_averageSi125[:]),'o-', label='Ge(hot)-r125',markerfacecolor='none', markeredgecolor=color_blind_safe[2],color=color_blind_safe[2])

plt.errorbar(sizeSi125[0], therm_averageSi125[0], yerrSi200b125, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[2],color=color_blind_safe[2])
plt.errorbar(sizeSi125[1], therm_averageSi125[1], yerrSi400b125, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[2],color=color_blind_safe[2]) 
plt.errorbar(sizeSi125[2], therm_averageSi125[2], yerrSi600b125, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[2],color=color_blind_safe[2]) 
plt.errorbar(sizeSi125[3], therm_averageSi125[3], yerrSi800b125, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[2],color=color_blind_safe[2]) 
plt.errorbar(sizeSi125[4], therm_averageSi125[4], yerrSi1000b125, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[2],color=color_blind_safe[2]) 

#Si250
plt.plot(sizeSi250[:], np.array(therm_averageSi250[:]),'o-', label='Ge(hot)-r250',markerfacecolor='none', markeredgecolor=color_blind_safe[3],color=color_blind_safe[3])

plt.errorbar(sizeSi250[0], therm_averageSi250[0], yerrSi200b250, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[3],color=color_blind_safe[3])
plt.errorbar(sizeSi250[1], therm_averageSi250[1], yerrSi400b250, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[3],color=color_blind_safe[3]) 
plt.errorbar(sizeSi250[2], therm_averageSi250[2], yerrSi600b250, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[3],color=color_blind_safe[3]) 
plt.errorbar(sizeSi250[3], therm_averageSi250[3], yerrSi800b250, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[3],color=color_blind_safe[3]) 
plt.errorbar(sizeSi250[4], therm_averageSi250[4], yerrSi1000b250, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[3],color=color_blind_safe[3]) 

#Si271
plt.plot(sizeSi271[:], np.array(therm_averageSi271[:]),'o-', label='Ge(hot)-r271',markerfacecolor='none', markeredgecolor=color_blind_safe[4],color=color_blind_safe[4])

plt.errorbar(sizeSi271[0], therm_averageSi271[0], yerrSi200b271, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[4],color=color_blind_safe[4])
plt.errorbar(sizeSi271[1], therm_averageSi271[1], yerrSi400b271, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[4],color=color_blind_safe[4]) 
plt.errorbar(sizeSi271[2], therm_averageSi271[2], yerrSi600b271, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[4],color=color_blind_safe[4]) 
plt.errorbar(sizeSi271[3], therm_averageSi271[3], yerrSi800b271, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[4],color=color_blind_safe[4]) 
plt.errorbar(sizeSi271[4], therm_averageSi271[4], yerrSi1000b271, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[4],color=color_blind_safe[4]) 

plt.legend(bbox_to_anchor=(1.1, 1.0), loc='upper left',labelspacing=1.0,fontsize=20) #,prop={'size': 20}
plt.xlabel('Length (nm)', fontdict={'fontsize':27})
plt.ylabel(r'Thermal conductivity (W/m·K)', fontdict={'fontsize':27})
#plt.title(r'Projection of Direct Convergence for all Systems', fontdict={'fontsize':27})

plt.show

In [ ]:
fig = plt.figure(figsize=(12,8))

#PristineGe
plt.plot(size_prstG[:], np.array(therm_average_prstG[:]),'s-.', label = 'Pristine Ge',markerfacecolor='none', markeredgecolor=color_blind_safe[7],color=color_blind_safe[7])

plt.errorbar(size_prstG[0], therm_average_prstG[0], yerr200b_prst, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[7],color=color_blind_safe[7])
plt.errorbar(size_prstG[1], therm_average_prstG[1], yerr400b_prst, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[7],color=color_blind_safe[7]) 
plt.errorbar(size_prstG[2], therm_average_prstG[2], yerr600b_prst, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[7],color=color_blind_safe[7]) 
plt.errorbar(size_prstG[3], therm_average_prstG[3], yerr800b_prst, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[7],color=color_blind_safe[7]) 
plt.errorbar(size_prstG[4], therm_average_prstG[4], yerr1000b_prst, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[7],color=color_blind_safe[7]) 

plt.legend(bbox_to_anchor=(1.1, 1.0), loc='upper left',labelspacing=1.0,fontsize=20) #,prop={'size': 20}
plt.xlabel('Length (nm)', fontdict={'fontsize':27})
plt.ylabel(r'Thermal conductivity (W/m·K)', fontdict={'fontsize':27})
#plt.title(r'Projection of Direct Convergence for all Systems', fontdict={'fontsize':27})

plt.show

In [ ]:
fig = plt.figure(figsize=(12,8))

#Straight GeSiGe
plt.plot(str_GeSiGe[:], np.array(therm_average_str_GeSiGe[:]),'s-.', label = 'Si|hot|',markerfacecolor='none', markeredgecolor=color_blind_safe[0],color=color_blind_safe[0])

plt.errorbar(str_GeSiGe[0], therm_average_str_GeSiGe[0], yerr200b_str_GeSiGe, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[0],color=color_blind_safe[0])
plt.errorbar(str_GeSiGe[1], therm_average_str_GeSiGe[1], yerr400b_str_GeSiGe, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[0],color=color_blind_safe[0])
plt.errorbar(str_GeSiGe[2], therm_average_str_GeSiGe[2], yerr600b_str_GeSiGe, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[0],color=color_blind_safe[0])
plt.errorbar(str_GeSiGe[3], therm_average_str_GeSiGe[3], yerr800b_str_GeSiGe, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[0],color=color_blind_safe[0])
plt.errorbar(str_GeSiGe[4], therm_average_str_GeSiGe[4], yerr1000b_str_GeSiGe, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[0],color=color_blind_safe[0])

plt.legend(bbox_to_anchor=(1.1, 1.0), loc='upper left',labelspacing=1.0,fontsize=20) #,prop={'size': 20}
plt.xlabel('Length (nm)', fontdict={'fontsize':27})
plt.ylabel(r'Thermal conductivity (W/m·K)', fontdict={'fontsize':27})
#plt.title(r'Projection of Direct Convergence for all Systems', fontdict={'fontsize':27})

plt.show

In [ ]:
fig = plt.figure(figsize=(12,8))

#Straight GeSiGe
plt.plot(str_GeSiGe[:], np.array(therm_average_str_GeSiGe[:]),'s-.', label = 'Si|hot|',markerfacecolor='none', markeredgecolor=color_blind_safe[0],color=color_blind_safe[0])

plt.errorbar(str_GeSiGe[0], therm_average_str_GeSiGe[0], yerr200b_str_GeSiGe, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[0],color=color_blind_safe[0])
plt.errorbar(str_GeSiGe[1], therm_average_str_GeSiGe[1], yerr400b_str_GeSiGe, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[0],color=color_blind_safe[0])
plt.errorbar(str_GeSiGe[2], therm_average_str_GeSiGe[2], yerr600b_str_GeSiGe, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[0],color=color_blind_safe[0])
plt.errorbar(str_GeSiGe[3], therm_average_str_GeSiGe[3], yerr800b_str_GeSiGe, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[0],color=color_blind_safe[0])
plt.errorbar(str_GeSiGe[4], therm_average_str_GeSiGe[4], yerr1000b_str_GeSiGe, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[0],color=color_blind_safe[0])


#Ge
#Ge50
plt.plot(sizeGe50[:], np.array(therm_averageGe50[:]),'s-.', ms=8, label='Si(hot)-r50',markerfacecolor='none', markeredgecolor=color_blind_safe[1], color=color_blind_safe[1])

plt.errorbar(sizeGe50[0], therm_averageGe50[0], yerrGe200b50, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[1], color=color_blind_safe[1])
plt.errorbar(sizeGe50[1], therm_averageGe50[1], yerrGe400b50, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[1], color=color_blind_safe[1]) 
plt.errorbar(sizeGe50[2], therm_averageGe50[2], yerrGe600b50, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[1], color=color_blind_safe[1]) 
plt.errorbar(sizeGe50[3], therm_averageGe50[3], yerrGe800b50, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[1], color=color_blind_safe[1]) 
plt.errorbar(sizeGe50[4], therm_averageGe50[4], yerrGe1000b50, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[1], color=color_blind_safe[1]) 


#Ge125
plt.plot(sizeGe125[:], np.array(therm_averageGe125[:]),'s-.', ms=8, label='Si(hot)-r125',markerfacecolor='none', markeredgecolor=color_blind_safe[2], color=color_blind_safe[2])

plt.errorbar(sizeGe125[0], therm_averageGe125[0], yerrGe200b125, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[2], color=color_blind_safe[2])
plt.errorbar(sizeGe125[1], therm_averageGe125[1], yerrGe400b125, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[2], color=color_blind_safe[2]) 
plt.errorbar(sizeGe125[2], therm_averageGe125[2], yerrGe600b125, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[2], color=color_blind_safe[2]) 
plt.errorbar(sizeGe125[3], therm_averageGe125[3], yerrGe800b125, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[2], color=color_blind_safe[2]) 
plt.errorbar(sizeGe125[4], therm_averageGe125[4], yerrGe1000b125, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[2], color=color_blind_safe[2]) 

#Ge250
plt.plot(sizeGe250[:], np.array(therm_averageGe250[:]),'s-.', ms=8, label='Si(hot)-r250',markerfacecolor='none', markeredgecolor=color_blind_safe[3],color=color_blind_safe[3])

plt.errorbar(sizeGe250[0], therm_averageGe250[0], yerrGe200b250, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[3],color=color_blind_safe[3])
plt.errorbar(sizeGe250[1], therm_averageGe250[1], yerrGe400b250, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[3],color=color_blind_safe[3]) 
plt.errorbar(sizeGe250[2], therm_averageGe250[2], yerrGe600b250, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[3],color=color_blind_safe[3]) 
plt.errorbar(sizeGe250[3], therm_averageGe250[3], yerrGe800b250, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[3],color=color_blind_safe[3]) 
plt.errorbar(sizeGe250[4], therm_averageGe250[4], yerrGe1000b250, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[3],color=color_blind_safe[3]) 

#Ge271
plt.plot(sizeGe271[:], np.array(therm_averageGe271[:]),'s-.', ms=8, label='Si(hot)-r271',markerfacecolor='none', markeredgecolor=color_blind_safe[4],color=color_blind_safe[4])

plt.errorbar(sizeGe271[0], therm_averageGe271[0], yerrGe200b271, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[4],color=color_blind_safe[4])
plt.errorbar(sizeGe271[1], therm_averageGe271[1], yerrGe400b271, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[4],color=color_blind_safe[4]) 
plt.errorbar(sizeGe271[2], therm_averageGe271[2], yerrGe600b271, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[4],color=color_blind_safe[4]) 
plt.errorbar(sizeGe271[3], therm_averageGe271[3], yerrGe800b271, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[4],color=color_blind_safe[4]) 
plt.errorbar(sizeGe271[4], therm_averageGe271[4], yerrGe1000b271, markersize=15, capsize=7, mfc='none',mec=color_blind_safe[4],color=color_blind_safe[4]) 


plt.legend(bbox_to_anchor=(1.1, 1.0), loc='upper left',labelspacing=1.0,fontsize=20) #,prop={'size': 20}
plt.xlabel('Length (nm)', fontdict={'fontsize':27})
plt.ylabel(r'Thermal conductivity (W/m·K)', fontdict={'fontsize':27})
#plt.title(r'Projection of Direct Convergence for all Systems', fontdict={'fontsize':27})


plt.show